In [ ]:
# SPDX-FileCopyrightText: © 2024 Tenstorrent AI ULC

# SPDX-License-Identifier: Apache-2.0
#
# Test "user experience" scenarios, i.e. different ways to use the API to run things on TT hardware
# Each test intentionally creates everything from scratch and uses no verification env, so that each
# of these tests can be used as user examples.
# There's also no verification of correctness of data, as that's not the point of these tests.
#
# All of these tests will run on silicon, in concurrent mode, by default. However, setting 
# PYBUDA_DEVMODE=1 env variable will drop them into Golden+sequential mode.

import queue
import torch
import pybuda
import pytest
from pybuda.config import _get_global_compiler_config

from pybuda.schedulers import LearningRateScheduler
from pybuda.pybudaglobal import pybuda_reset
from pybuda._C.backend_api import BackendDevice, BackendType, DeviceMode 
from test.utils import download_model

# https://github.com/pytorch/pytorch/wiki/Autograd-and-Fork
mp_context = torch.multiprocessing.get_context('spawn')

def _safe_read(q):
    """
    Read a queue, but return None if an error was raised in the meantime, preventing a hang on error.
    """
    while True:
        try:
            data = q.get(timeout = 0.5)
            return data
        except queue.Empty as _:
            if pybuda.error_raised():
                raise RuntimeError("Error raised in pybuda")
        except KeyboardInterrupt:
            return None

# Sample PyBuda module. 이 PyBudaTestModule 단독으로는 아무 연산도 수행하지 않으며, 이는 PyBUDA의 기능을 시험하거나 다른 테스트/추론/학습 흐름에 투입하기 위한 "선언적 정의" 임.
class PyBudaTestModule(pybuda.PyBudaModule): # class 클래스를 정의한다는 키워드. '이름 붙여서(PyBudaTestModule을 말함) 하나의 설계도를 만들겠다'는 뜻. 따라서 실체 물건(객체)는 없는 상태.
    def __init__(self, name): # __init__ => 생성자constructor(클래스(설계도)를 기반으로 객체를 만들 때 자동으로 실행되는 특별한 함수). 객체의 초기 상태를 설정하는 역할. self는 이 클래스에서 만들어지는 실제 객체 자신"을 가리키는 예약어
        super().__init__(name)  # super()는 부모 클래스(상속한 클래스. 여기에서는 pybuda.PyBudaModule)의 기능을 불러오는 역할. 부모 클래스(pybuda.PyBudaModule) 기능을 먼저 불러온 후, 내 기능을 얹는 것
        self.weights1 = pybuda.Parameter(torch.rand(32, 32), requires_grad=True)
        self.weights2 = pybuda.Parameter(torch.rand(32, 32), requires_grad=True)

    def forward(self, act1, act2):
        m1 = pybuda.op.Matmul("matmul1", act1, self.weights1) # act1, act2 매 실행마다 사용자가 넣는 입력 데이터(예: 이미지 데이터, 문장 벡터, 센서 값 등)
        m2 = pybuda.op.Matmul("matmul2", act2, self.weights2) # 'Pre-Trained AI Model'을 사용하면, weights1과 weights2가 '사전 학습된 모델 로드 (pretrained weights)'를 사용함.
        return m1 + m2, m2 # class PyBudaTestModule(pybuda.PyBudaModule): => 이 모델이 m1 + m2, m2를 최종 출력으로 씀. 학습/디버깅/분석용: 중간값도 같이 봐야 할 수 있어서 두 개의 값으로 반환

# Sample PyBuda module  # 실행 환경: Tenstorrent NPU, 목적: PyBUDA 모델 작성 및 NPU 테스트, PyBUDA는 연산자 그래프로 분해하여 NPU에 최적화 실행
class PyBudaTestModuleOneOut(pybuda.PyBudaModule): # 이 클래스(설계도)가 출력값이 하나만 있는 버전이라는 걸 다른 사용자(개발자)에게 알림. PyBudaTestModuleOneOut은 추론-only 버전. 학습/디버깅/분석 필요없음때 사용 의미. 다른 사람이 볼 때 가독성 높이기 위해 사용한 개발자 편의용 라벨
    def __init__(self, name):
        super().__init__(name)
        self.weights1 = pybuda.Parameter(torch.rand(32, 32), requires_grad=True)
        self.weights2 = pybuda.Parameter(torch.rand(32, 32), requires_grad=True)

    def forward(self, act1, act2):
        m1 = pybuda.op.Matmul("matmul1", act1, self.weights1)
        m2 = pybuda.op.Matmul("matmul2", act2, self.weights2)
        return m1 + m2  # 실제 추론 배포용: 최종 결과만 필요할 때. 하나의 값으로 반환

# Sample PyBuda module  # Transformer 모델(입력을 벡터로 바꾼 후 전체 관계를 계산하는 AI 구조)의 핵심 연산 중 하나인 Self-Attention(문장 안에서 어떤 단어에 집중할지 자동으로 계산)의 Query-Key 계산을 PyBUDA로 구현한 예제. Value Weights는 선언되어 있지만 실제 연산에 사용하지 않았음을 주목하자. 이유는 이 모델(PyBudaTestQueryKeyModule)은 "Qeury-Key Attention Score 계산"만 보여주는 간단한 예제이기 때문. Value weights는 실제 모델에서는 반드시 사용됨. 이 예제는 학습용 또는 디버깅용 예제로 일부 연산만 보여 준 것임.
class PyBudaTestQueryKeyModule(pybuda.PyBudaModule):
    def __init__(self, name, hidden_dim = 128, num_heads = 4):  # hidden_dim = 128(“히든 차원”이라는 뜻으로, 벡터 길이를 나타냄. 가볍고 안정적인 테스트용, 실무에서도 자주 사용. 학습할 때도, 추론할 때도 모두 사용, NPU 서버 프리세일즈 관점: test는 128, PoC는 256-512 점진 확장. 고객 요구사항(정확도 vs 속도) 듣고 조정, 실제 추론 부하는 NPU 자원 평가와 함께 검증), mun_heads = 4 (Multi-Head Attention의 Head의 갯수. 4개의 Head 즉 attention(가중치)을 병렬로 동시에 수행하겠다는 뜻. Head1 문법적관계(주어-동사), Head2 시간정보(과거-현재), Head3 감정뉘앙스(긍정-부정), Head4 주체흐름(단어연결구조), 4값 빠른테스트, 4~8 PoC/데모, 8 실서비스, 12~16 고품질서비스(법률,금융 등등))
        super().__init__(name)
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads

        self.key_weights = pybuda.Parameter(torch.rand(1, 1, hidden_dim, hidden_dim), requires_grad=True)   # Self-Attention(가중치) 계산을 위해 각 입력 벡터(예: 토큰 임베딩)를 세 가지로 나눔. key란?내가 가진 정보는 이런 특징이야-검색 대상. 키를 계산할 학습 가능한 가중치 생성(초기값은 무작위)(AI 모델이 하는 것임 사람이 하는게 아니라)
        self.query_weights = pybuda.Parameter(torch.rand(1, 1, hidden_dim, hidden_dim), requires_grad=True) # Qeury란?나는 지금 어떤 정보를 찾고 싶다 - 질문 역할. 쿼리를 위한 가중치 생성(AI 모델이 하는 것임 사람이 하는게 아니라)
        self.value_weights = pybuda.Parameter(torch.rand(1, 1, hidden_dim, hidden_dim), requires_grad=True) # Value란?실제 내용물-최종 결과에 반영될 값. 값을 위한 가중치 생성(AI 모델이 하는 것임 사람이 하는게 아니라)

    def forward(self, encoder_input):   # 이 모델(PyBudaTestQueryKeyModule)이 입력을 받아서 어떤 연산을 수행할지 정의하는 부분. encoder_input(의미를 갖고 있는 백터 인풋)
        query = pybuda.op.Matmul(f"mha_query", encoder_input, self.query_weights)   # f"mha_query"는 Python의 f-string 문법. "mha_query"는 그냥 이 행렬곱셈 연산(Matmul; Matrix Multiplication)에 이름을 붙여주는 String(Label)임. 즉, "이 행렬 곱 연산의 이름을 'mha_query'로 부르자"는 의미. 왜 이름을 붙이나?(연산추적,시각화도구에서그래프구성,테스트나로그에서식별자역할). mha는 Multi-Head Attention의 약자. 행렬곱셈을 통해 전체 정보 계산
        query = pybuda.op.HSlice(f"mha_query_slice", query, self.num_heads) # HSlice(PyBUDA에서 쓰이는 특수 연산. 이름 그대로 가로(Horizontal) 방향으로 자른다는 의미. 즉, 텐서(입력된 다차원 벡터, hidden_dim = 128)를 num_heads = 4 만큼 가로로 나누는 연산. 즉, query, key, value 벡터를 num_heads개로 가로 분할(split) 하는 연산. 나누는 이유: Multi-Head Attention 구조 때문에 Hidden_dim = 128을 4로 나눠 4개(32씩) 분할 되어 각기 다른 관점에서 attention을 계산하기 위해서 임). hidden_dim = 128 즉 벡터를 head 수(4) 만큼 쪼개어 Multi-Head Attention 계산 해 냄.

        key = pybuda.op.Matmul(f"mha_key", encoder_input, self.key_weights)
        key = pybuda.op.HSlice(f"mha_key_slice", key, self.num_heads)
        key = pybuda.op.Transpose(f"mha_key_transpose", key, 2, 3)  # Self-Attention 계산을 가능하게 만드는 핵심 연산 중 하나. 텐서(다차원 배열)의 차원(축, axis)을 서로 바꾸는 연산(예를 들어, 2번 차원과 3번 차원을 맞바꾸는 것). 왜 위치를 바꾸는가?(행과 열을 바꿔야 올바른 곱셈이 가능)

        attention_scores = pybuda.op.Matmul(f"mha_as", query, key)  # 위에서 나오는 2개의 query와 3개의 key 값이 다시 벡터 값이 되고 이를 행렬곱셈 연산으로 최종 attention_scores가 나오는 것.
        return attention_scores


class PyBudaTestForkWithThreeUsers(pybuda.PyBudaModule):    # PyBudaTestForkWithThreeUsers: PyBudaTest(이 클래스가 실제 모델 구현보다는 테스트용(test case) 으로 만들어졌다는 걸 암시), Fork(PyBUDA 연산 그래프 상에서 입력(encoder_input)이 세 개의 다른 연산 (Matmul)으로 분기(fork) 되는 구조이기 때문에 사용된 이름), WithThreeUsers("Users"는 입력 텐서를 사용하는 연산자(operator) 들을 뜻함. encoder_input이 다음 세 연산자(mm_a, mm_b, mm_c)에게 동시에 사용됨. 즉, 이 입력을 사용하는 사용자(user)가 세 명이라는 의미). "간단한 행렬 연산(Matmul + Add) 구조를 테스트"한다는 것은 단순한 연산 그 자체보다는 PyBUDA 컴파일러와 하드웨어 시스템 전체가 제대로 작동하는지를 검증하는 데 목적이 있음(데이터 흐름 최적화(Fork) 검증, PyBUDA 연산자 그래프 처리 정상 여부 확인, 파라미터 학습 가능성(Gradient Flow) 검증, Tenstorrent 하드웨어 매핑 테스트, PyBUDA Scheduler와 Compiler 정상 작동 여부 확인)
    def __init__(self, name, hidden_dim = 128, num_heads = 4):
        super().__init__(name)
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads

        self.mm_a_weights = pybuda.Parameter(torch.rand(1, 1, hidden_dim, hidden_dim), requires_grad=True)  # 학습 가능한 파라미터(가중치 텐서)를 생성하는 구문, torch.rand(1, 1, hidden_dim, hidden_dim); Pytorch의 랜덤 텐서 생성 함수. 1, 1 (배치사이즈, num_heads). 배치사이즈: 한 번에 모델에 입력되는 데이터 샘플의 개수를 말함.(예; 1 한번에 이미지 1장 처리, 32 한번에 이미지 32장 처리. NPU 카드 및 서버 성능에 따라 숫자 늘릴 수 있음. Tenstorrent Loudbox의 경우 NPU 카드 4장일 시 32~128 가능능), num_heads의 경우 4인데도 불구하고 예제에서는 1만 사용되었는데, 단일 헤드로만 먼저 테스트하려는 의도로 보임. 테스트 예제라는 것임. 
        self.mm_b_weights = pybuda.Parameter(torch.rand(1, 1, hidden_dim, hidden_dim), requires_grad=True)  # pybuda.Parameter(...); pybuda에서 이 텐서를 학습 대상 파라미터로 등록
        self.mm_c_weights = pybuda.Parameter(torch.rand(1, 1, hidden_dim, hidden_dim), requires_grad=True)  # requires_grad=True; 역전파(backpropagation) 과정에서 이 파라미터에 대해 기울기(gradient) 를 계산하게 해 줌. 즉, 학습 대상임을 명시하는 플래그

    def forward(self, encoder_input):
        a = pybuda.op.Matmul(f"mm_a", encoder_input, self.mm_a_weights)
        b = pybuda.op.Matmul(f"mm_b", encoder_input, self.mm_b_weights)
        c = pybuda.op.Matmul(f"mm_c", encoder_input, self.mm_c_weights)

        add_a_b = pybuda.op.Add(f"add_a_b", a, b)               # 텐서 간 덧셈. 같은 shape(모양; 텐서의 각 차원의 크기)이거나 broadcasting(서로 다른 shape을 가진 텐서끼리 연산할 때, 자동으로 크기를 맞춰주는 규칙) 가능한 경우 가능. Broadcasting이 가능한 조건(텐서의 뒤에서부터 차원을 맞춰 나감, 같은 차원이거나 한 쪽이 1인 경우, 그 차원은 자동으로 확장. 실전예시: a.shape = (32, 1, 10, 128) b.shape = (32, 1, 10, 128) ADD 가능 / a.shape = (32, 1, 10, 128) b.shape = (1, 1, 10, 128) b의 1이 32로 확장(Broadcasting)되어 두 텐서 크기가 맞아서 add가 가능하게 됨)
        add_a_b_c = pybuda.op.Add(f"add_a_b_c", add_a_b, c)     # PyBUDA에서는 기본적으로 Add 연산자(Binary Operator)가 2개의 입력만 받기 때문에 a + b + c를 직접 한 줄로 쓸 수 없어 두 개씩 나눠서 순차적으로 더해야 함(연산(노드) 분리. 그래픽,병렬화,디버깅,시각화 위해 필수). 또한 디버깅/시각화 목적에도 유리(연산자 이름(add_a_b, add_a_b_c)으로 그래프 추적이 쉬움. 각 단계별 출력을 따로 확인 가능). Tenstorrent NPU는 내부 연산 그래프를 기반으로 컴파일하기 때문이라도 명확한 연산 노드 분리가 필요
        return add_a_b_c



# Sample PyTorch module. 실행환경: CPU 또는 GPU (PyTorch)
class PyTorchTestModule(torch.nn.Module):   # torch.nn.Module(모델(모듈)을 만들 때 사용하는 기본 클래스)
    def __init__(self): # 
        super().__init__()
        self.weights1 = torch.nn.Parameter(torch.rand(32, 32), requires_grad=True)  # torch.nn.Parameter(학습 가능한 파라미터(=weight)), torch.rand(32, 32)(32x32짜리 랜덤 숫자 행렬; 딥러닝에서는 모델의 가중치(weight) 들을 학습을 통해 조금씩 조정해가며 정답을 예측. 학습을 시작할 때는 아직 아무것도 모르는 상태이기 때문에 일단은 무작위(random) 값으로 시작. 이렇게 시작한 뒤, 학습(Training)을 통해 이 랜덤 값이 의미 있는 값으로 바뀌어 감. 32×32는 단순한 우연이 아니라, 딥러닝 모델의 복잡도와 NPU/GPU 하드웨어 효율에 맞춘 일반적인 최소 단위 크기. 32x32 자주 사용하는 이유: 표현력(Expressiveness)이 충분, NPU/GPU의 병렬 처리 단위에 최적화, 딥러닝 모델 구조에서 흔히 쓰는 크기)
        self.weights2 = torch.nn.Parameter(torch.rand(32, 32), requires_grad=True)  # requires_grad=True(학습 중에 파라미터가 업데이트 되도록 함, backpropagation (역전파) 과 직접적으로 관련된 설정. 역전파(Backpropagation)를 통해 기울기(gradient)를 계산하겠다는 뜻). weight1, weight2 2개만 지정 이유: 두 개의 입력(act1, act2)에 대해 각각 하나씩 대응하는 가중치(weights1, weights2)를 적용하기 때문에 딱 2개의 가중치만 정의한 것

    def forward(self, act1, act2):  # forward() 입력 데이터를 넣었을 때 실제 연산이 일어나는 함수(순전파). act1, act2는 전처리된 입력 데이터 (Preprocessed Input) 혹은 이전 레이어의 출력(activation)
        m1 = torch.matmul(act1, self.weights1)  # 행렬 곱 (Matrix Multiplication)
        m2 = torch.matmul(act2, self.weights2)
        return m1 + m2, m1  # 중간 결과도 함께 확인하거나 사용할 경우. m1 + m2; 최종 연산 결과 (합계). m1; 	act1 × weights1 결과 (중간값), 만일 m2도 있다면, act2 × weights2 결과 (중간값)이 됨. m1 혹은 m2까지 반환계산 이유: 디버깅 및 로깅 용도, 실험적 분석 용도, 후속 계산에서 재사용. 주의: 반환된 값이 많아질수록 모델 인터페이스가 복잡해 짐.

# Sample PyTorch module. PyTorch로 작성된 3개의 간단한 신경망 모듈 클래스. 전체적으로 모델 설계와 출력 처리, 손실 함수(loss function)의 흐름을 설명하려는 예제
class PyTorchTestModuleOneOut(torch.nn.Module): # torch.nn.Module(모델(모듈)을 만들 때 사용하는 기본 클래스). 이름의 "OneOut"은 출력이 하나라는 걸 의미
    def __init__(self):
        super().__init__()
        self.weights1 = torch.nn.Parameter(torch.rand(32, 32), requires_grad=True)  # torch.nn.Parameter; 학습 가능한 가중치(weight)를 정의하는 방법. torch.rand(32, 32); 무작위 숫자로 채운 32×32 행렬을 생성
        self.weights2 = torch.nn.Parameter(torch.rand(32, 32), requires_grad=True)  # requires_grad=True; 역전파(backpropagation)할 때 이 가중치를 학습하겠다는 의미

    def forward(self, act1, act2):  # 두 입력(act1, act2)을 받아 각각 행렬 곱 후, 결과를 더해 반환
        m1 = torch.matmul(act1, self.weights1)
        m2 = torch.matmul(act2, self.weights2)
        return m1 + m2  # 최종 결과만 필요한 경우. 출력이 하나라는 걸 의미

class PyTorchTestModuleOneInputAndOneOut(torch.nn.Module):  # 이 모듈은 입력이 하나, 출력도 하나
    def __init__(self):
        super().__init__()
        self.weights = torch.nn.Parameter(torch.rand(32, 32), requires_grad=True)   # 32×32 학습 가능한 가중치
    
    def forward(self, act): # 단일 입력 (예: 하나의 텐서)
        m = torch.matmul(act, self.weights) # act와 weights를 행렬 곱 (즉, 특징 변환)
        return m

class PyTorchLoss(torch.nn.Module): # 커스텀 손실 함수(Loss Function)
    def forward(self, input):
        return input.sum()  # 텐서의 모든 값을 더해서 하나의 숫자로 반환

#
# Run inference on module directly. PyBUDA와 PyTorch 모듈을 각각 실행(inference)해보는 예제. 각 함수는 입력 데이터를 생성하고, 모델을 실행한 뒤, 결과를 출력
#
def test_module_direct_pybuda():    # 모델을 한번 실행(inference pass), PyBUDA 모델 실행
    input1 = torch.rand(4, 32, 32)  # 배치 크기 (Batch size) → 입력 샘플 4개를 한 번에 처리하겠다는 뜻. 각 텐서 크기 32X32
    input2 = torch.rand(4, 32, 32)  # 두 번째 입력도 동일 크기로 생성

    # Run single inference pass on a PyBuda module directly
    output = PyBudaTestModule("direct").run(input1, input2) # 이 코드는 PyTorch가 아닌, PyBUDA 프레임워크용 객체를 직접 생성하는 것이고, Tenstorrent의 NPU에서 직접 실행 가능한 그래프를 구성하는 클래스 인스턴스. 생성한 입력을 넣고 모델을 한번 실행. "direct"는 이 모듈을 구별하기 위한 이름(tag) 역할
    print(output)   # 결과를 출력

def test_module_direct_pytorch():   # 모델을 한번 실행(inference pass), PyTorch 모델을 PyBUDA로 감싼 후 실행
    input1 = torch.rand(4, 32, 32)  # 배치 크기 (Batch size) → 입력 샘플 4개를 한 번에 처리하겠다는 뜻. 각 텐서 크기 32X32
    input2 = torch.rand(4, 32, 32)  # 두 번째 입력도 동일 크기로 생성. 

    # Run single inference pass on a PyTorch module, using a wrapper to convert to PyBuda first
    output = pybuda.PyTorchModule("direct_pt", PyTorchTestModule()).run(input1, input2) # PyTorchTestModule(); 앞에서 정의한 PyTorch 모듈 클래스 객체 생성. pybuda.PyTorchModule("direct_pt", PyTorchTestModule()); 이 PyTorch 모듈을 PyBUDA에서 실행 가능하도록 감쌈(wrap). "direct_pt"는 이 모듈을 구별하기 위한 이름(tag) 역할
    print(output)   # 결과를 출력

#
# Run inference through run_inference without placing on device. PyBUDA 환경에서 추론(inference)을 실행하는 예제. 차이점은 하나는 PyBUDA 전용 모델, 다른 하나는 PyTorch 모델을 PyBUDA로 감싼 형태
#
def test_run_inference_direct_pybuda(): # PyBUDA로 직접 만든 모델을 사용해 추론 실행
    input1 = torch.rand(4, 32, 32)  # 무작위 입력 데이터를 생성 (배치 4개)
    input2 = torch.rand(4, 32, 32)  # 무작위 입력 데이터를 생성 (배치 4개)

    # Run inference on a PyBuda module, with given inputs
    inputs = {"act2" : input2, "act1" : input1} # PyBUDA 모델의 입력값 이름에 맞게 딕셔너리로 정리
    output_q = pybuda.run_inference(PyBudaTestModule("run_direct"), inputs=[inputs])    # pybuda.run_inference()는 즉시 추론만 수행하는 함수(=디바이스에 올리지 않고 추론만 실행하는 "간단한 실행"). PyBudaTestModule(...); PyBUDA로 직접 만든 모델을 생성. pybuda.run_inference(...); 해당 모델을 이용해 추론 실행 (그래프를 컴파일하고 NPU에 할당하지는 않음)
    output = _safe_read(output_q)   # 출력값을 안전하게 읽음 (queue에서 꺼냄).
    print(output)   # 결과 출력
    # _safe_read(output_q) 안전하게 꺼낸다는 의미
    # 아직 NPU에서 계산이 완전히 끝나지 않아서 결과가 안 들어왔는데, get()으로 꺼내려 하면, 프로그램이 멈추거나 오류남.
    # _safe_read(); 큐가 비어 있지 않을 때만 꺼냄. 결과가 들어올 때까지 살짝 기다렸다가 꺼냄을 의미.

def test_run_inference_direct_pytorch():    # PyTorch 모델을 PyBUDA가 감싸서 추론 실행
    input1 = torch.rand(4, 32, 32)  # 무작위 입력 데이터를 생성 (배치 4개)
    input2 = torch.rand(4, 32, 32)  # 무작위 입력 데이터를 생성 (배치 4개)

    # Run inference, using a wrapper to convert PyTorch module to PyBuda, and with given inputs
    inputs = {"act2" : input2, "act1" : input1}
    output_q = pybuda.run_inference(pybuda.PyTorchModule("run_direct_pt", PyTorchTestModule()), inputs=[inputs])    # pybuda.run_inference()는 즉시 추론만 수행하는 함수(=디바이스에 올리지 않고 추론만 실행하는 "간단한 실행"). PyTorchTestModule(); PyTorch로 만든 모델 클래스 객체. pybuda.PyTorchModule(...); 	PyTorch 모델을 PyBUDA에서 쓸 수 있게 감쌈. 
    output = _safe_read(output_q)   # 출력값을 안전하게 읽음 (queue에서 꺼냄)
    print(output)   # 결과 출력


#
# Run inference by placing on device first. Tenstorrent NPU에 모델과 입력을 올려서(로딩) 추론(inference)을 실행하는 전체 절차를 보여주는 예제. 실제 디바이스 환경을 연동하는 구조
#
def test_run_inference_placed_pybuda(): # PYBUDA 전용 모델 사용
    input1 = torch.rand(4, 32, 32)
    input2 = torch.rand(4, 32, 32)

    # Create a TT device
    tt0 = pybuda.TTDevice("tt0")    # TTDevice("tt0"); Tenstorrent NPU 장치를 소프트웨어적으로 지정

    # Place a module on the device
    tt0.place_module(PyBudaTestModule("placed"))    # place_module(); PyBuda 모델을 디바이스에 올림 (배치/컴파일). "placed"; 이름(String) 태그 임.

    # Push intputs to the device
    tt0.push_to_inputs((input1, input2))    # push_to_inputs(); 생성한 입력(input1, input2) 데이터를 디바이스에 전달

    # Run pipeline, and read the outputs
    output_q = pybuda.run_inference()   # 추론 실행 (디바이스에서 실제로 계산 시작)
    output = _safe_read(output_q)   # 결과 큐에서 출력값을 꺼내서 확인
    print(output) # 결과 출력

def test_run_inference_placed_pytorch():    # PyTorch 모델을 PyBUDA가 감싸서 실행
    input1 = torch.rand(4, 32, 32)
    input2 = torch.rand(4, 32, 32)

    # Create a TT device
    tt0 = pybuda.TTDevice("tt0")    # TTDevice("tt0"); Tenstorrent NPU 장치를 소프트웨어적으로 지정

    # Place a module on the device, using a wrapper to convert PyTorch module to PyBuda
    tt0.place_module(pybuda.PyTorchModule("placed_pt", PyTorchTestModule()))    # place_module(); PyTorch 모델을 디바이스에 올림 (배치/컴파일)
    
    # Push intputs to the device
    tt0.push_to_inputs((input1, input2))    # push_to_inputs(); 생성한 입력(input1, input2) 데이터를 디바이스에 전달

    # Run pipeline, and read the outputs
    output_q = pybuda.run_inference()   # 추론 실행 (디바이스에서 실제로 계산 시작)
    output = _safe_read(output_q)   # 결과 큐에서 출력값을 꺼내서 확인
    print(output)   # 결과 출력

#
# Repeated calls to run inference on the same module. 같은 모델 또는 같은 디바이스에 대해 추론(inference)을 반복해서 실행하는 두 가지 방식의 예제. 즉, "같은 모듈을 다시 컴파일하지 않고 여러 번 실행하는 방법"을 학습하기 위한 코드. "재컴파일 없이 반복 실행"이 중요한 이유: 속도 향상(컴파일은 시간이 오래 걸림. 한 번만 하고 재활용하면 훨씬 빠름), 실시간 응답(반복 입력에 대해 빠르게 처리 가능), 리소스 절약(중복된 그래프 생성 방지, 메모리 절감)
#
def test_module_direct_repeated():  # PyBUDA 모듈을 Python 내에서 반복 실행
    module = PyBudaTestModule("direct") # PyBudaTestModule이라는 PyBUDA 기반 모델을 하나 생성. 이름 "direct"는 그냥 구분용 문자열(String)

    # Run on given inputs. 입력(input1, input2) 2개를 생성하고 .run()을 통해 첫 번째 추론을 실행
    input1 = torch.rand(4, 32, 32)  # 배치 크기 (Batch size) → 입력 샘플 4개를 한 번에 처리하겠다는 뜻. 각 텐서 크기 32X32
    input2 = torch.rand(4, 32, 32)
    output = module.run(input1, input2) # module.run(...); PyBUDA 모듈을 직접 실행 (캐시된 컴파일 결과 사용)
    print(output)   # 결과를 출력

    # Run again, without recompiling. 다시 실행해도 모델을 재컴파일하지 않는다는 뜻. 내부적으로 PyBUDA는 이미 컴파일된 상태를 캐시하고 있기 때문에 빠르게 반복 실행이 가능
    input1 = torch.rand(4, 32, 32)
    input2 = torch.rand(4, 32, 32)
    output = module.run(input1, input2) # module.run(...); PyBUDA 모듈을 직접 실행 (캐시된 컴파일 결과 사용)
    print(output)

    # Run again, without recompiling. 이런 구조를 총 3번 반복
    input1 = torch.rand(4, 32, 32)
    input2 = torch.rand(4, 32, 32)
    output = module.run(input1, input2) # module.run(...); PyBUDA 모듈을 직접 실행 (캐시된 컴파일 결과 사용)
    print(output)

def test_run_inference_placed_repeated():   # 디바이스에 모델을 올리고 여러 입력을 차례대로 실행
    input1 = torch.rand(4, 32, 32)
    input2 = torch.rand(4, 32, 32)
    tt0 = pybuda.TTDevice("tt0")    # 디바이스 및 모델 초기화. 가상 NPU 디바이스를 생성 ("tt0").
    tt0.place_module(PyBudaTestModule("placed"))    # 디바이스 및 모델 초기화. 모델(PyBudaTestModule)을 디바이스에 올림(배치/로드)(1회만).

    # Push one input and run. 첫 번째 추론
    tt0.push_to_inputs((input1, input2))    # push_to_inputs(...); 입력 전송. 입력 텐서를 디바이스에 보냄.
    output_q = pybuda.run_inference()   # 추론 실행

    output = _safe_read(output_q)   # 결과를 안전하게 읽고
    print(output)   # 결과 출력

    # Push two more inputs, and run one more time on both inputs, without recompiling. 입력 2쌍 추가 → 2번 연속 실행
    for _ in range(2):  # 3번 중 첫 번째는 위에서 이미 실행했으므로, 남은 2개를 반복 처리하는 구조
        input1 = torch.rand(4, 32, 32)
        input2 = torch.rand(4, 32, 32)
        tt0.push_to_inputs((input1, input2))    # push_to_inputs(...); 새 입력 2쌍을 디바이스 입력 큐에 추가. 입력 텐서를 디바이스에 보냄.

    pybuda.run_inference(input_count=2) # input_count=2은 “큐에 올라간 입력 2개 추론해 주세요”라는 뜻. 왜 명시적으로 input_count를 써야 하나? PyBUDA는 입력 큐에서 여러 개의 입력이 대기할 수 있기 때문에, run_inference(input_count=N)을 통해 몇 개만 처리할지 정확히 지정해야 함. 즉, 입력 큐의 크기와 실제 실행 횟수를 분리해서 관리할 수 있도록 설계된 구조

    for _ in range(2):
        output = _safe_read(output_q)   # 출력 큐에서 2개의 결과를 하나씩 읽어옵
        print(output)   # 결과 출력


#
# Run inference through setup + run_forward calls. Tenstorrent NPU에서 추론을 더 정밀하게 제어하는 방법 예제. 
# setup(초기화, initialize_pipeline()) + run_forward(실행) 조합을 사용하는 예제 (사용이유: 성능 향상, 반복 실행 최적화, 대규모 처리, 개발 유연성 확보)
def test_setup_forward_calls():
    tt0 = pybuda.TTDevice("tt0")    # 디바이스 생성
    tt0.place_module(PyBudaTestModule("placed"))    # 사용자 정의 PyBUDA 모듈인 PyBudaTestModule("placed")을 디바이스에 올림

    # Compile & initialize the pipeline for inference, with given shapes. 
    output_q = pybuda.initialize_pipeline(training=False, sample_inputs=(torch.rand(4, 32, 32), torch.rand(4, 32, 32))) # 파이프라인 초기화 (initialize_pipeline) 및 컴파일 실행. training=False; 학습 모드가 아닌 추론 모드로 컴파일하겠다는 의미. sample_inputs=(...); 추론에 사용될 입력 샘플의 텐서 shape을 지정 → 이 정보로 연산 그래프를 구성함. output_q; 출력 결과가 저장될 큐(queue) 객체 반환
        
    # Push & run_forward manually. 입력 2회 넣고 각각 추론 실행 (run_forward)
    for _ in range(2):  # 과정을 2번 반복
        input1 = torch.rand(4, 32, 32)  # 입력 데이터 생성 (배치 크기 4, 32x32 차원)
        input2 = torch.rand(4, 32, 32)  # 입력 데이터 생성 (배치 크기 4, 32x32 차원)
        tt0.push_to_inputs((input1, input2))    # 입력 데이터를 NPU로 전송
        pybuda.run_forward(input_count=1)   # 한 번의 추론 실행 (입력 1쌍에 대해 실행)

        print(_safe_read(output_q)) # 결과 큐에서 결과를 꺼내서 출력
    
    # 왜 initialize_pipeline() + run_forward()를 쓰는가? 명시적 제어(컴파일과 실행을 분리해서 제어할 수 있음), 성능 최적화(반복 수행 시 컴파일 1번 + 다회 실행 구조(for _ in range(1000):) 가능), 대규모 시스템 확장(복수 디바이스, 스트리밍 등에서 더 유연하게 활용 가능), 학습 지원(향후 학습(Training) API와 같은 구조로 확장 가능)
    # 왜 “성능 최적화”가 되는가? initialize_pipeline()에서 딱 한 번만 그래프 컴파일과 장치 초기화를 함. 이후 반복 run_forward()에서는 빠르게 연산만 수행함. 즉, 반복 실행 구조를 만들 수 있으면서도 오버헤드(컴파일과 장치 초기화)는 한 번만 발생


#
# Run inference in concurrent mode, then push more inputs afterwards (won't work on Golden). Tenstorrent NPU에서 "지연 입력(Push Delayed)" 구조로 추론하는 방법 예제. 즉, 입력을 먼저 일부만 넣고, 나머지는 나중에 넣는 방식으로도 추론을 수행할 수 있음.
# 'Golden이란': Tenstorrent PyBUDA 또는 TT-NN 소프트웨어 개발 및 테스트 환경에서 사용되는 정확도 검증 모드. Golden 모드는 "완전히 동기(synchronous)적이고 결정적(deterministic)"인 실행만 허용. 목적: 기준(reference) 실행 결과와 비교하여 정확도(accuracy)를 검증하는 데 사용. 이 예제는 지연(push delayed) 즉 비동기(asynchronous) 방식이 있어서, Golden 모드 검증 기준에는 부합되지 않음. 그래서 "Golden에서는 동작하지 않음(won't work on Golden)"이라고 주석이 달려 있는 것.
def test_run_inference_delayed_push():  # 테스트 함수 시작
    
    #### Skip the test on golden. Golden 환경에서는 실행 안 되도록 예외 처리. "PYBUDA_DEVMODE" 환경변수가 존재하면 Golden이 아님.
    import os
    if "PYBUDA_DEVMODE" in os.environ:  # os.environ은 파이썬의 os 모듈에 포함된 환경변수 딕셔너리(dictionary). 이 딕셔너리를 통해 현재 운영체제에 설정된 환경 변수들을 조회하거나 수정할 수 있음. "PYBUDA_DEVMODE" 있으면, Golden 환경이 아니므로 나머지 코드가 실행
        pytest.skip()
    ####

    tt0 = pybuda.TTDevice("tt0")    # Tenstorrent NPU 디바이스 "tt0"를 만들고,
    tt0.place_module(PyBudaTestModule("placed"))    # 여기에 PyBuda 모듈 PyBudaTestModule("placed")를 배치

    input1 = torch.rand(4, 32, 32)
    input2 = torch.rand(4, 32, 32)
    tt0.push_to_inputs((input1, input2))    # 입력 1개만 먼저 Push. 배치 하나 분량(4)의 입력 텐서(32x32)를 먼저 1개만 전송

    # Run with input count 3, although only one is pushed
    output_q = pybuda.run_inference(input_count=3)  # 3개의 입력을 처리하라고 명령 (하지만 위 1개만 함). 아직 입력은 1개밖에 없지만, "3개 처리하라"는 추론 요청. PyBUDA는 백그라운드에서 기다림 (스레드가 입력 올 때까지 대기)

    # Read one output that should've been produced
    output = _safe_read(output_q)   # 첫 번째 결과는 바로 나옴
    print(output)   # 결과 출력

    # The inference thread is running in the background, waiting for data. Let's push two more.
    for _ in range(2):  # 나머지 2개 입력을 나중에 Push. '_' 반복 변수의 이름. 
        input1 = torch.rand(4, 32, 32)
        input2 = torch.rand(4, 32, 32)
        tt0.push_to_inputs((input1, input2))    # 이제 나머지 2개의 입력을 순차적으로 전송

    # Read two more outputs
    for _ in range(2):
        output = _safe_read(output_q)   # 2번째와 3번째 입력에 대한 출력값을 읽음
        print(output)   # 결과 출력

    # 지연 입력(Push Delayed) 구조의 장점:
    # 실시간 데이터 처리에 적합 (Streaming Inference. 먼저 받은 입력(실시간음성,비디오)부터 추론을 시작하고, 나머지 입력은 나중에 오더라도 중단 없이 계속 수행)
    # NPU 자원 효율 극대화(한 입력 처리 후 바로 다음 입력이 들어오면, 연산 리소스 낭비 없이 연속 추론 가능)
    # 입력 대기 큐를 활용한 유연한 처리 구조(run_inference(input_count=N) 호출 후, 아직 입력이 부족하더라도 백그라운드에서 대기하고 있기 때문에 사용자는 언제고 입력을 동적으로 밀어넣을 수 있음)
    # 동기/비동기 모델 혼합 실행 가능(기존 방식은 모든 입력이 준비된 후 시작하는 동기적(synchronous) 방식. 이 방식은 비동기적(asynchronous) 흐름도 가능하게 만들어줌)
    # PyBUDA의 추론 파이프라인 테스트/디버깅에 유용(NPU 내부 큐/스케줄링 시스템이 제대로 작동하는지 확인 가능, 입력 부족 상태에서의 행동, 지연 처리 타이밍 등을 실험할 수 있음)
    # 유용한 환경: 
    # 영상 스트리밍 추론: 프레임 도착이 랜덤해도 추론은 멈추지 않음
    # 멀티 유저 시스템: 각 사용자 입력이 시간차로 들어오더라도 계속 추론 가능
    # 센서 처리 시스템: IoT 센서 데이터가 순차적으로 들어오는 구조
    

#
# Run inference on multiple devices - combinations of cpu / tt device. PyBUDA를 이용한 다중 디바이스(NPU, CPU) 추론(inference)과 학습(training) 테스트 예제
#
def test_cpu_tt_pipeline(): # CPU + TT(NPU) 조합으로 추론. 명시적으로 분리된 코드, 디버깅이 쉬운 코드

    cpu0 = pybuda.CPUDevice("cpu0") # cpu0: CPU 상에서 PyTorch 모듈 실행
    cpu0.place_module(pybuda.PyTorchModule("stage0", PyTorchTestModule()))  # 각 디바이스에 서로 다른 stage를 배치함
    tt1 = pybuda.TTDevice("tt1")    # tt1: NPU(TT) 상에서 PyBUDA 모듈 실행
    tt1.place_module(PyBudaTestModule("stage1"))    # 각 디바이스에 서로 다른 stage를 배치함

    input1 = torch.rand(4, 32, 32)
    input2 = torch.rand(4, 32, 32)
    cpu0.push_to_inputs((input1, input2))   # 입력을 CPU 쪽에 전달

    output_q = pybuda.run_inference()   # 내부적으로 stage0(→CPU) → stage1(→TT) 순서로 연산됨
    print(_safe_read(output_q))

def test_cpu_tt_pipeline_compact(): # 위 코드와 같지만, 장치 생성 시 모듈을 직접 바인딩. 간결한 코드, 반복 코드를 줄이고 싶을 때

    cpu0 = pybuda.CPUDevice("cpu0", module=pybuda.PyTorchModule("stage0", PyTorchTestModule())) # 생성자에서 바로 place_module() 생략. 간결하게 같은 동작
    tt1 = pybuda.TTDevice("tt1", module=PyBudaTestModule("stage1")) # 생성자에서 바로 place_module() 생략. 간결하게 같은 동작

    input1 = torch.rand(4, 32, 32)
    input2 = torch.rand(4, 32, 32)
    cpu0.push_to_inputs((input1, input2))   # 입력을 CPU 쪽에 전달

    output_q = pybuda.run_inference()
    print(_safe_read(output_q)) # 내부적으로 stage0(→CPU) → stage1(→TT) 순서로 연산됨

# Run training, read back checkpoints and loss
def test_training_read_back():  # TT(NPU)에서 학습 실행 + 체크포인트와 손실(loss) 읽기. 체크포인트란? 모델의 현재 상태(가중치 등)를 저장해 둔 스냅샷(필요 이유: 학습 도중 끊겨도 중간부터 다시 시작 가능, 학습 중 가장 성능 좋은 모델만 골라 저장 가능, 학습 완료 후 checkpoint를 로딩해서 추론(inference) 실행 가능)
    pybuda.config.set_configuration_options(
            default_df_override=pybuda.DataFormat.Float16_b,    # 연산 정밀도(데이터포맷)를 Float16으로 설정
    )
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestModuleOneOut("module"))   # TT(NPU) 위에 모델 배치
    tt0.place_loss_module(pybuda.op.loss.L1Loss("l1_loss")) # TT(NPU) 위에 손실 함수 배치함. L1Loss는 딥러닝에서 자주 사용되는 손실 함수(loss function) 중 하나. 두 값(예측값과 실제값)의 차이의 절댓값(absolute difference)을 모두 더해서 평균 낸 손실 함수. 보통 Mean Absolute Error (MAE)라고도 불림. 특징: 에러에 덜 민감(극단적인 오차(outlier)에 크게 반응하지 않음), 스파스(sparse) 모델 유도(가중치에 제약을 주면, 일부 가중치가 0으로 수렴할 가능성 ↑, 불필요한 가중치(weight)를 0으로 줄여서 모델을 더 가볍고 효율적으로 만든다는 뜻, "어설픈 값보다는 아예 0으로 만드는 게 낫다"), 미분 불연속점 존재(0에서 기울기가 정의되지 않아 일부 최적화에서 불편할 수 있음 (하지만 대부분 문제 없음))

    loss_q = mp_context.Queue()
    checkpoint_q = mp_context.Queue()

    input1 = torch.rand(4, 32, 32)  # i배치 크기 4 (batch size = 4)는 전처리된 데이터가 한 번에 4개 들어간다는 의미
    input2 = torch.rand(4, 32, 32)
    tt0.push_to_inputs((input1, input2))
    tt0.push_to_target_inputs(torch.rand(4, 32, 32))    # push_to_target_inputs: 학습용 정답(레이블) 전달. 학습용 정답(레이블): 모델이 맞춰야 할 정답 데이터, 즉 모델이 예측한 결과와 비교할 기준이 되는 데이터를 뜻함. 정답(레이블, Target)이란?: 모델이 입력(input)을 보고 예측(prediction)을 할 때, 그것이 맞았는지 틀렸는지를 판단하기 위해 필요한 기준값. 이 기준값을 흔히 "레이블(label)" 또는 "타겟(target)"이라고 부름.(예. 강아지 사진(이미지 input), 실제정답(Target, 레이블) -> "dog" (1)). 모델은 사진을 보고 강아지라고 예측. 그 결과를 레이블과 비교해서 손실(loss) 계산. 그 손실을 바탕으로 역전파(backpropagation) 수행

    pybuda.run_training(checkpoint_queue = checkpoint_q, loss_queue=loss_q) # 학습 실행 후 checkpoint_q, loss_q에 결과 저장

    print("checkpoint: ", _safe_read(checkpoint_q)) # _safe_read()로 출력값 확인. Python에서 함수나 변수 이름 앞에 밑줄(_)이 붙으면, 그건 "내부용(private)" 함수임을 암시
    print("loss: ", _safe_read(loss_q)) # _safe_read()로 출력값 확인. Python에서 함수나 변수 이름 앞에 밑줄(_)이 붙으면, 그건 "내부용(private)" 함수임을 암시

# Run training pipeline, with loss on CPU, read back checkpoints and loss
#@pytest.mark.skip(reason="Intermittent hangs on silicon")
def test_training_pipeline_read_back(): # TT에서 모델 실행, CPU에서 손실 계산하는 학습 파이프라인
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestModule("stage0")) # 모델은 TT에서 실행
    cpu1 = pybuda.CPUDevice("cpu1", module=pybuda.PyTorchModule("stage1", PyTorchTestModuleOneOut()))
    cpu1.place_loss_module(pybuda.PyTorchModule("l1loss", torch.nn.L1Loss()))   # 손실 함수는 CPU에서 실행(이유: 연산량이 적기 때문(CPU에서 처리해도 성능 차이 없음, NPU 리소스를 보호(텐서 연산용으로 양보), 디버깅과 유지보수가 쉬움(CPU에서 하면 값 추적(디버깅)이 편하고 안정적))) → Cross-device(서로 다른 종류의 디바이스(예: CPU + NPU 또는 CPU + GPU)가 한 모델 파이프라인을 나눠서 실행하는 구조를 말함) 학습 파이프라인 구성

    loss_q = mp_context.Queue()
    checkpoint_q = mp_context.Queue()

    input1 = torch.rand(4, 32, 32)
    input2 = torch.rand(4, 32, 32)
    tt0.push_to_inputs((input1, input2))    # 데이터는 각각의 디바이스로 푸시

    cpu1.push_to_target_inputs(torch.rand(4, 32, 32))   # 데이터는 각각의 디바이스로 푸시

    pybuda.run_training(checkpoint_queue = checkpoint_q, loss_queue=loss_q)

    print("checkpoint: ", _safe_read(checkpoint_q)) # 학습 후 checkpoint 읽음
    print("loss: ", _safe_read(loss_q)) # 학습 후 loss 읽음


#
# Run inference pipeline on a Transformers model. BERT 기반 Transformer 모델을 PyBUDA로 실행하는 예제로, CPU → TT(NPU) 두 단계를 거쳐 추론(inference) 파이프라인을 구축하는 과정을 보여 줌.
#
def test_transformers_pipeline_inference():

    from transformers import BertModel, BertTokenizer   # Hugging Face(인터넷)의 사전학습 모델과 토크나이저 사용. BertModel: 사전학습된 BERT 모델 구조. BertTokenizer: 문장을 토큰(token)으로 쪼개주는 역할

    tokenizer = download_model(BertTokenizer.from_pretrained, "prajjwal1/bert-tiny")    # 토크나이저 다운로드와 입력 문장 준비. prajjwal1/bert-tiny: 원래 BERT는 크기가 커서 느림. 가볍고 빠른 BERT 변형 모델 (Tiny BERT). 실험이나 에지 디바이스에서 자주 사용. .from_pretrained 호출하면 내부적으로 Hugging Face 라이브러리(transformers)가 실행되고, 기본 도메인(Hugging Face)에 HTTP 요청을 보냄
    input_sentence = "BERT is a transformers model pretrained on a large corpus of English data in a self-supervised fashion. This means it was pretrained on the raw texts only, with no humans labelling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those texts. More precisely, it was pretrained with two objectives: Masked language modeling (MLM): taking a sentence, the model randomly masks 15% of the words in the input then run the entire masked sentence through the model and has to predict the masked words. This is different from traditional recurrent neural networks (RNNs) that usually see the words one after the other, or from autoregressive models like GPT which internally mask the future tokens. It allows the model to learn a bidirectional representation of the sentence." # 입력 문장
    input_tokens = tokenizer.encode(input_sentence, max_length=128, pad_to_max_length=True) # 긴 문장을 128 토큰 길이로 변환하고 패딩 처리. 패딩 처리: NLP(자연어 처리)에서 문장의 길이를 고정된 토큰 수로 맞추기 위해 부족한 부분을 채우는 작업을 말함. 머신러닝 모델(특히 Transformer 계열)은 입력의 형상(shape)이 동일하게 고정되어 있어야 함. 여러 입력을 같은 길이로 맞춰서 모델에 일괄 처리 가능하게 하기 위함. 예시문장. "나는 사과를 좋아해." 토큰 개수 5. max_length가 128일 때 패딩 결과는 [실제 5개 토큰] + [0 × 123개]. 입력 문장의 토큰 길이가 5개밖에 안 된다면, 128개로 맞추기 위해 나머지 123개를 [PAD] 토큰(보통 숫자 0)으로 채움.

    model = download_model(BertModel.from_pretrained, "prajjwal1/bert-tiny", torchscript=False, add_pooling_layer=False)    # 모델 다운로드 및 분리. BertModel.from_pretrained(...): pretrained 모델 불러오기. torchscript=False: TorchScript 최적화는 사용하지 않음 (PyBUDA 직접 실행용) 즉, 이건 "모델을 그냥 원래 모습 그대로 받아와!" 라는 뜻.불필요하게 변환하지 말고, PyBUDA가 직접 처리할 수 있게 생긴 상태로 가져와 라는 말. torchscript=True로 하면, "모델을 속도 빠르게 하려고, 딱딱한 구조로 바꿔 줄게!"라는 뜻인데, 이렇게 하면 좋을 수도 있지만, 유연성이 떨어져 PyBUDA가 못 알아볼수 있음. // add_pooling_layer=False: Pooling 생략 (encoder만 사용 목적 (문장의 각 단어(토큰) 에 대한 벡터 연산(표현)만 뽑고 싶을 때 or 다른 모델[BERT Encoder → CNN → 분류기]과 조합할 때 or RAG, Retrieval, QnA 시스템에서 BERT 인코더만 써서 문장 임베딩 만들고, 나중에 다른 블록으로 처리하는 구조를 만들고 싶을 때). 여기서 Pooling(풀링)이란 건 뭐냐면, 여러 토큰의 출력들을 하나로 요약(압축)해주는 과정. Polling 필요한 이유: 예를 들어, 문장을 분류하거나 감정 분석(예. 부정적인지 긍정적인지)하려면, 문장의 전체 의미만 필요하니까 Pooling을 해서 문장 전체를 하나로 요약. 따라서 감정 분석하기 위해 Pooling이 True 되면 됨. 필요 없으면 False로 하면 됨. 이 예제에서 add_pooling_layer=False로 설정한 이유는 사용 목적이 분류(classification)가 아니기 때문. 이 예제의 목적은 BERT 인코더만 실행해서, 각 입력 토큰에 대한 임베딩 벡터를 얻는 것임.
    cpu0 = pybuda.CPUDevice("cpu0", module=pybuda.PyTorchModule("bert_embeddings", model.embeddings))   # 파이프라인(BERT의 구조를 여러 디바이스(CPU, NPU 등)에 나눠서 실행되도록 연결하는 전체 처리 흐름을 의미. 구성이유: 병렬처리최적화, CPU/NPU 역할분담) 구성. CPU 디바이스에 Embedding Layer(계산량이 적기 때문에 CPU에. 이유: Embedding은 단순히 토큰 ID를 벡터로 매핑하는 작업) 배치. 이것을 이해하려면, BERT의 구조([입력문장] -> [임베딩] -> [인코더(각 단어(토큰) 에 대한 벡터 표현만 뽑고 싶을 때)] -> [풀링(감정분석)] -> [분류기])를 알아야 함.
    tt0 = pybuda.TTDevice("tt1", module=pybuda.PyTorchModule("bert_encoder", model.encoder))    # 파이프라인(BERT의 구조를 여러 디바이스(CPU, NPU 등)에 나눠서 실행되도록 연결하는 전체 처리 흐름을 의미. 구성이유: 병렬처리최적화, CPU/NPU 역할분담) 구성. TT NPU 디바이스에 Encoder Layer(계산량이 많음. Self-Attention, LayerNorm, Dense 연산 등 매우 복잡한 연산들) 배치.

    cpu0.push_to_inputs(torch.Tensor(input_tokens).int().unsqueeze(0))  # 입력 전달 및 추론 실행. torch.Tensor(...).unsqueeze(0) → [1, 128] 형태로 배치 차원 추가. push_to_inputs() → 첫 번째 CPU Stage로 입력 전달.
    output_q = pybuda.run_inference()   # run_inference() → 전체 파이프라인 추론 실행. 

    print(_safe_read(output_q)) # _safe_read(output_q) → 결과 출력

    # tensor.unsqueeze(0) 의미
    # input_tokens = [101, 1045, 2293, 102, 0, 0, ..., 0]  # 길이 128짜리 토큰 ID 리스트. 이것을 텐서로 바꾸면, torch.Tensor(input_tokens).shape = [128] 즉 1차원 벡터 (토큰 128개)
    # 머신러닝 모델에 데이터를 넣을 때는 "배치(batch)" 차원이 있어야 함. 즉, 몇 개의 문장을 동시에 처리할 것인지를 나타내는 축
    # [128] ← 1개의 문장 (1차원)
    # [1, 128] ← 1개 문장을 배치 1로 만든 것 (2차원)
    # 그래서 unsqueeze(0)을 쓰면:
    # torch.Tensor(input_tokens).unsqueeze(0).shape = [1, 128] -> "문장 1개짜리 배치 1개" 형태로 만들어 줌
    # 정리하면, [128]	문장 1개 (배치 정보 없음)
    # [1, 128]	문장 1개짜리 배치 (모델에 넣을 수 있는 형태)
    # 배치 이해 예시. (모델에 넣기 위해 배치 차원을 앞에 추가하기 위해 사용하는 것이 unsqueeze(0))
    # input = [101, 1045, 2293, 102]  # 길이 4짜리 1D 토큰 리스트
    # tensor = torch.tensor(input)   # → shape: [4]
    # tensor = tensor.unsqueeze(0)   # → shape: [1, 4]

#
# Run inference pipeline on a Transformers model, enabling cpu fallback on unsupported ops. BERT 기반 Transformer 모델을 PyBUDA 환경에서 실행하되, 일부 연산이 NPU에서 지원되지 않으면 CPU로 대신 처리하는 fallback(폴백) 기능을 활성화해서 사용하는 예제
# BERT 모델을 Tenstorrent NPU (TTDevice)에서 실행. NPU가 처리 못하는 연산(op)이 있으면 CPU에서 자동 처리 (fallback). 총 5번의 추론을 반복 실행
def test_transformers_pipeline_fallback_inference():

    from transformers import BertModel, BertTokenizer   # Hugging Face에서 BERT 모델과 토크나이저 가져옴.

    compiler_cfg = pybuda.config._get_global_compiler_config()  # PyBUDA 내부 설정을 가져오는 코드. fallback 설정 등을 여기에서 조정 가능 (이 예제 코드에서는 fallback 직접 설정은 안 하고, 단순히 컴파일러 설정 객체(config object)를 가져오기만 함). compiler_cfg.enable_cpu_fallback = True을 다음 줄에 추가 해 주어야 함.
    # compiler_cfg.enable_cpu_fallback = True

    tokenizer = download_model(BertTokenizer.from_pretrained, "prajjwal1/bert-tiny")    # "prajjwal1/bert-tiny"는 Hugging Face에 올라온 작은 BERT 모델 토크나이저. 이걸 다운로드해서 사용할 준비를 함.
    input_sentence = "BERT is a transformers model pretrained on a large corpus of English data in a self-supervised fashion. This means it was pretrained on the raw texts only, with no humans labelling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those texts. More precisely, it was pretrained with two objectives: Masked language modeling (MLM): taking a sentence, the model randomly masks 15% of the words in the input then run the entire masked sentence through the model and has to predict the masked words. This is different from traditional recurrent neural networks (RNNs) that usually see the words one after the other, or from autoregressive models like GPT which internally mask the future tokens. It allows the model to learn a bidirectional representation of the sentence."    # 테스트용 긴 영어 문장 하나 입력으로 준비함.
    input_tokens = tokenizer.encode(input_sentence, max_length=128, pad_to_max_length=True) # 이 문장을 최대 128개의 숫자 토큰으로 변환하고, 128보다 짧으면 0으로 패딩(상기 예제 참고)해서 항상 [1, 128] 형태의 "고정 길이 텐서"로 만듦.

    model = download_model(BertModel.from_pretrained, "prajjwal1/bert-tiny", torchscript=False, add_pooling_layer=False)    # 모델 다운로드 및 분리. BertModel.from_pretrained(...): pretrained 모델 불러오기. torchscript=False: TorchScript 최적화는 사용하지 않음 (PyBUDA 직접 실행용) 즉, 이건 "모델을 그냥 원래 모습 그대로 받아와!" 라는 뜻.불필요하게 변환하지 말고, PyBUDA가 직접 처리할 수 있게 생긴 상태로 가져와 라는 말. torchscript=True로 하면, "모델을 속도 빠르게 하려고, 딱딱한 구조로 바꿔 줄게!"라는 뜻인데, 이렇게 하면 좋을 수도 있지만, 유연성이 떨어져 PyBUDA가 못 알아볼수 있음. // add_pooling_layer=False: Pooling 생략 (encoder만 사용 목적 (문장의 각 단어(토큰) 에 대한 벡터 연산(표현)만 뽑고 싶을 때 or 다른 모델[BERT Encoder → CNN → 분류기]과 조합할 때 or RAG, Retrieval, QnA 시스템에서 BERT 인코더만 써서 문장 임베딩 만들고, 나중에 다른 블록으로 처리하는 구조를 만들고 싶을 때). 여기서 Pooling(풀링)이란 건 뭐냐면, 여러 토큰의 출력들을 하나로 요약(압축)해주는 과정. Polling 필요한 이유: 예를 들어, 문장을 분류하거나 감정 분석(예. 부정적인지 긍정적인지)하려면, 문장의 전체 의미만 필요하니까 Pooling을 해서 문장 전체를 하나로 요약. 따라서 감정 분석하기 위해 Pooling이 True 되면 됨. 필요 없으면 False로 하면 됨. 이 예제에서 add_pooling_layer=False로 설정한 이유는 사용 목적이 분류(classification)가 아니기 때문. 이 예제의 목적은 BERT 인코더만 실행해서, 각 입력 토큰에 대한 임베딩 벡터를 얻는 것임.
    tt0 = pybuda.TTDevice("tt0", module=pybuda.PyTorchModule("bert", model))    # NPU 디바이스(tt0)에 PyTorch 모델을 PyBUDA 형식으로 감싸서 배치 → 이렇게 하면 모델이 Tenstorrent NPU에서 실행 가능해짐.



    for i in range(5):  # 이 블록은 추론을 5번 반복함:
        tt0.push_to_inputs(torch.Tensor(input_tokens).int().unsqueeze(0))   # unsqueeze(0); [128]를 → [1, 128]로 바꿔서 배치(1 부분이 배치) 추가. 이렇게 해 줘야 하는 이유; 사용하는 모델에서 배치(batch) 처리를 해 줘야 하기 때문 // push_to_inputs()	입력을 디바이스에 전달
        output_q = pybuda.run_inference()   # run_inference(); 추론 수행 (NPU에서 실행하되, 지원 안 되는 연산은 CPU가 자동 처리)
        print(_safe_read(output_q)) # 결과 출력 (큐에서 출력값 꺼냄)

    # CPU 연산이 필요한 이유; Tenstorrent의 NPU (Neural Processing Unit) 와 PyBUDA 소프트웨어 스택이 아직 개발/성장 단계에 있어서, 모든 종류의 연산(Operation) 을 NPU에서 직접 실행할 수 없는 상황을 의미

#
# Run training through setup + manual loop of fwd/bwd/opt.  BERT 모델로 학습(training)하는 과정을 수동 루프(manual loop)로 구현한 예제. Manual Loop는 학습 과정에서의 forward / backward / optimizer step을 사용자가 직접 하나씩 코드로 명시해서 실행한다는 의미. 이런 방식은 사용자가 학습 흐름을 직접 제어할 수 있게 해주지만, 자동 루프보다 코드가 길고 복잡해질 수 있음. 손실값(Loss)을 줄이기 위해 역전파(backward)를 반복적으로 수행하면서 모델 파라미터를 점점 최적화(Optimization)해 나가는 구조
# 사전 훈련된 BERT 모델을 불러옴 -> PyBUDA용으로 디바이스 설정 -> 손실 함수(Loss function) 설정 -> 입력 데이터를 생성 -> 학습 루프 실행: forward → backward → optimizer 순서
def test_training_manual_loop_with_cpu_fallback():  # 함수 정의. 테스트 목적이며, 수동 학습 루프(manual training loop)를 CPU fallback 기능과 함께 사용
    from transformers import BertForMaskedLM, BertTokenizer, BertConfig # Hugging Face의 transformers 라이브러리에서 BERT 관련 클래스들을 가져옴. BertForMaskedLM: 마스킹 언어 모델 / BertTokenizer: 입력 문장을 토크나이즈 / BertConfig: 설정값을 정의하는 클래스

    config = download_model(BertConfig.from_pretrained, "prajjwal1/bert-tiny")  # "prajjwal1/bert-tiny"라는 이름의 사전 학습된 BERT 모델 설정(config) 을 Hugging Face 모델 허브에서 다운로드 받아서, BertConfig 형식으로 래핑(wrap)해서 사용하겠다는 의미. Bertconfig 사용 이유: 모델의 내부 설정(config)을 명시적으로 제어하거나, 그 설정을 기반으로 모델 객체를 직접 생성하고 싶기 때문. 이 예제 코드에서는 prajjwal1/bert-tiny에 사전 정의된 config.json 내용을 그대로 받아서 사용. 즉, 사용자가 별도로 설정을 바꾸지는 않은 상태. 사용자가 코드 형태로 임의 설정하여 사용할 수도 있음. 이렇게 하면 Hugging Face 모델 허브에서 다운받지 않고, 사용자가 직접 지정한 구조대로 BERT 모델을 생성할 수 있게 됨
    model = BertForMaskedLM(config) # 문장 속에 가려진 단어(=MASK) 를 맞추는 모델. Bert; BERT 모델 – 문장의 문맥을 양쪽(좌우)에서 이해하는 똑똑한 언어 모델 // For: ~을 위한 // MaskedLM: Masked Language Modeling, 즉 가려진 단어 예측 모델. 예시 문장: "나는 오늘 아침에 [MASK]을 마셨다." MASK 예측결과는 "커피", "우유", "물" 등일 수 있음. 중요 이유: LLM은 문장을 제대로 이해해야 함. (문장 분류, 질의 응답, 감정 분석, 번역)
    tt0 = pybuda.TTDevice("tt0", module=pybuda.PyTorchModule("bert", model), optimizer=pybuda.optimizers.SGD(learning_rate=0.1, device_params=True)) # PyBUDA 디바이스 tt0를 생성 ("tt0": 디바이스 이름 // PyTorchModule: PyTorch 모델(bert)을 PyBUDA에 연결 // SGD 옵티마이저: 학습률 0.1, device_params=True는 디바이스에서 파라미터를 관리). SGD; Stochastic Gradient Descent (확률적 경사 하강법)의 줄임말. 딥러닝에서 가장 기본적인 최적화 방법. 오차를 줄이는 방향(gradient 방향)으로 조금씩 파라미터를 조정해 가며 학습. learning_rate=0.1; 얼마나 많이 움직일지 결정하는 숫자. 값이 크면 빨리 움직이지만, 너무 크면 목적지를 지나칠 수 있음. 값이 작으면 안정적이지만 느림. 여기서 0.1은 다소 빠르게 파라미터를 조정하는 설정. device_params=True; 모델의 파라미터(weight 등)를 NPU 디바이스(tt0) 안에 올려서 관리하겠다는 뜻. 기본적으로 PyTorch는 CPU나 GPU에서 파라미터를 관리. True로 지정하면, 파라미터가 NPU에 직접 올라가서 연산 및 업데이트가 거기서 일어남. NPU에서 모든 연산을 효율적으로 처리할 수 있어서 속도 및 대역폭 절감 효과가 나타남.
    tt0.place_loss_module(pybuda.PyTorchModule("CEL", torch.nn.CrossEntropyLoss())) # 손실 함수(loss function) 로 CrossEntropyLoss를 PyBUDA 디바이스에 연결. "CEL"은 이름 (Cross Entropy Loss의 약자)

    sample_inputs = (torch.randint(config.vocab_size, (1,128)) ,)   # 학습 샘플 데이터 생성 (튜토리얼용 랜덤 입력). sample_inputs: 1개의 문장, 128개의 토큰을 랜덤 생성
    sample_targets = (torch.rand(1, config.vocab_size) ,)   # 학습 샘플 데이터 생성 (튜토리얼용 랜덤 입력). 1개의 타겟, vocab_size 만큼의 확률값 (실제로는 클래스 레이블이어야 함)

    checkpoint_q = pybuda.initialize_pipeline(  # PyBUDA 학습 파이프라인 초기화
            training=True,  # training=True: 학습 모드
            sample_inputs=sample_inputs,    # 입력 샘플 기반으로 그래프 초기 컴파일
            sample_targets=sample_targets)  # 입력 샘플 기반으로 그래프 초기 컴파일


    for step in range(2):   # 수동 학습 루프 (manual loop). 총 2 step 반복
        for acc_step in range(2):   # 각 step 당 2번의 gradient accumulation 수행
            tt0.push_to_inputs(torch.randint(config.vocab_size, (1,128)))   # 새로운 입력 문장 (1×128) 을 만들어 디바이스에 푸시
            tt0.push_to_target_inputs(torch.rand(1, config.vocab_size).long())  # 새로운 타겟 데이터 (클래스) 도 디바이스에 푸시
            pybuda.run_forward(input_count = 1) # 순전파 (forward pass) 실행
            pybuda.run_backward(input_count = 1, zero_grad = (acc_step == 0))   # 역전파 (backward pass) 실행. acc_step == 0일 때만 gradient를 초기화(zero_grad) 이는 gradient accumulation(누적) 전략
    # 실제 실행 흐름
    #   step    acc_step    forward     backward    optimizer
    #   0       0           ^           ^
    #   0       1           ^           ^
    #                                               ^
    #   1       0           ^           ^
    #   1       1           ^           ^
    #                                               ^
    # 모델 파라미터 = 등산로에서 내가 서 있는 위치
    # 손실 함수 (Loss) = 산의 고도 (낮을수록 좋음)
    # 기울기 (Gradient) = 고도가 낮아지는 방향 (내리막 방향). 딥러닝에서 gradient(기울기)가 거의 수평(0에 가까움)이 되도록 만드는 것이 최적화의 목표 중 하나
    # 옵티마이저(optimizer) = 그 방향으로 내려가는 방법 (속도, 경로)

        pybuda.run_optimizer(checkpoint=True)   # 2번의 acc_step이 끝나면 최적화(optimizer step) 실행

# Run training through run_training without placing on device
# Run training by placing on device first
# Repeated calls to run training
# Run training in concurrent mode, then push inputs afterwards
# Run training in concurrent mode, read checkpoints as they come out
# Run inference on multiple devices - combinations of cpu / tt device

#
# Run training through setup + manual loop of fwd/bwd/opt. 하나는 NPU(고속 연산 장치), 하나는 CPU로 나누어 모델을 실행하는 구조. 
#
def test_training_manual_loop():    # AI 모델 학습을 직접 수동으로 제어하는 예제 함수. 자동으로 학습하지 않고, 사용자가 한 단계씩 "직접 시킨다"는 구조

    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestModule("stage0"), optimizer=pybuda.optimizers.SGD(learning_rate=0.1, device_params=True)) # NPU(tt0) 준비. PyBudaTestModule("stage0"): 테스트용 인공지능 모델 모듈. SGD 옵티마이저: 학습률 0.1, device_params=True는 디바이스에서 파라미터를 관리). SGD; Stochastic Gradient Descent (확률적 경사 하강법)의 줄임말. 딥러닝에서 가장 기본적인 최적화 방법. 오차를 줄이는 방향(gradient 방향)으로 조금씩 파라미터를 조정해 가며 학습. learning_rate=0.1; 얼마나 많이 움직일지 결정하는 숫자. 값이 크면 빨리 움직이지만, 너무 크면 목적지를 지나칠 수 있음. 값이 작으면 안정적이지만 느림. 여기서 0.1은 다소 빠르게 파라미터를 조정하는 설정. device_params=True; 모델의 파라미터(weight 등)를 NPU 디바이스(tt0) 안에 올려서 관리하겠다는 뜻. 기본적으로 PyTorch는 CPU나 GPU에서 파라미터를 관리. True로 지정하면, 파라미터가 NPU에 직접 올라가서 연산 및 업데이트가 거기서 일어남. NPU에서 모든 연산을 효율적으로 처리할 수 있어서 속도 및 대역폭 절감 효과가 나타남.
    cpu1 = pybuda.CPUDevice("cpu1", module=pybuda.PyTorchModule("stage1", PyTorchTestModuleOneOut()),   # CPU(cpu1) 준비. stage1: 또 다른 모델 조각. 이 CPU 모델은 PyTorch 기반. 
            optimizer_f = lambda m: torch.optim.SGD(m.parameters(), lr=0.5))    # optimizer_f = lambda ...: 학습 방법을 지정 (학습률 0.5로 SGD 사용)
    cpu1.place_loss_module(pybuda.PyTorchModule("l1loss", torch.nn.L1Loss()))   # CPU 디바이스에 오차 계산 함수(Loss function) 를 추가. L1Loss: 예측과 정답이 얼마나 다른지 숫자로 계산하는 함수
    
    # Compile & initialize the pipeline for training, with given shapes
    input1 = torch.rand(4, 32, 32)  # 입력 데이터 만들기. 배치 크기 (Batch size) → 입력 샘플 4개를 한 번에 처리하겠다는 뜻. 각 텐서 크기 32X32
    input2 = torch.rand(4, 32, 32)  # 입력 데이터 만들기. 배치 크기 (Batch size) → 입력 샘플 4개를 한 번에 처리하겠다는 뜻. 각 텐서 크기 32X32
    checkpoint_q = pybuda.initialize_pipeline(  # PyBUDA 파이프라인 초기화: "이 모델을 학습시키겠다"는 뜻. 체크포인트란? 모델의 현재 상태(가중치 등)를 저장해 둔 스냅샷(필요 이유: 학습 도중 끊겨도 중간부터 다시 시작 가능, 학습 중 가장 성능 좋은 모델만 골라 저장 가능, 학습 완료 후 checkpoint를 로딩해서 추론(inference) 실행 가능)
            training=True, 
            sample_inputs=(input1, input2), # 예시 입력 (input1과 input2)
            sample_targets=(torch.rand(4, 32, 32),))    # 실제 의미 있는 정답이 아닌 ‘랜덤값 정답’. 이 코드는 실제 성능을 높이기 위한 모델 훈련이 아니기 때문에 PyBUDA 파이프라인이 잘 작동하는지, multi-device 연산이 잘 되는지, optimizer가 돌면서 파라미터가 바뀌는지 이런 걸 확인하기 위한 테스트 코드. 따라서  "정답 역할을 하는 값은 필요하지만, 꼭 진짜일 필요는 없음" → 랜덤값 사용함.


    for step in range(2):   # 학습 루프. 전체 학습 반복 횟수 (2번)
        for acc_step in range(2):   # acc_step: 작은 학습 단위 (gradient 누적) 반복 2번. 즉, 한 번의 학습(step) 안에 두 번의 계산(accumulation)이 있음
            tt0.push_to_inputs((input1, input2))    # NPU(tt0)에 입력 데이터를 전달
            cpu1.push_to_target_inputs(torch.rand(4, 32, 32))   # CPU에 실제 의미 있는 정답이 아닌 ‘랜덤값 정답’ 전달

            pybuda.run_forward(input_count = 1) # 순전파 (forward pass): 모델이 예측을 수행
            pybuda.run_backward(input_count = 1, zero_grad = (acc_step == 0))   # 역전파 (backward pass): 오차에 따라 모델을 개선할 방향 계산. zero_grad = (acc_step == 0): 처음이면 gradient 초기화. acc_step == 0일 때: 처음 시작이니 gradient 초기화. acc_step == 1일 때: 이전 단계에서 계산된 gradient를 유지한 채 누적함
            # 처음 줄(acc_step=0): 새 종이 꺼내서 쓰기 시작 (zero_grad=True)
            # 두 번째 줄(acc_step=1): 종이에 이미 쓴 내용 위에 추가로 계산 적음 (zero_grad=False)
            # 작성 다 되면: 종이 제출 (optimizer step)

        pybuda.run_optimizer(checkpoint=True)   # 두 번의 forward/backward가 끝난 후, 진짜로 모델을 업데이트함. checkpoint=True: 결과를 체크포인트로 저장

    print("Checkpoint: ", _safe_read(checkpoint_q)) # 학습 결과(출력값)를 읽어서 콘솔에 출력

# 항목                  첫번째코드(test_training_manual_loop_with_cpu_fallback)                 두 번째 코드(test_training_manual_loop)
# 주석제목               동일(Run training through setup + manual loop of fwd/bwd/opt)          동일(Run training through setup + manual loop of fwd/bwd/opt)
# 모델                  Hugging Face BertForMaskedLM 사용                                      커스텀 테스트 모듈 (PyBudaTestModule, PyTorchTestModuleOneOut) 사용
# 디바이스구성           하나의 TT 디바이스 (tt0)만 사용                                           TT 디바이스(tt0) + CPU 디바이스(cpu1) 동시 사용
# 연산분산(스테이지분리)  없음 (모두 한 디바이스에서 처리)                                           모델을 두 스테이지로 나눠 각기 다른 디바이스에서 실행
# 손실함수위치           tt0에 CrossEntropyLoss 배치                                             cpu1에 L1Loss 배치
# 입력구조              단일 텐서 입력 (torch.randint(...))                                      두 개의 입력 텐서 (input1, input2)
# 대상구조(Target)      단일 클래스 예측용 텐서                                                   입력과 같은 모양의 회귀(regression) 대상 텐서
# CPU Callback         암묵적으로 존재 (NPU가 못 하면 CPU에서 수행)                                명시적으로 CPUDevice를 따로 사용


#
# Run training through setup + manual loop of fwd/bwd, while copying final gradients. 최적화(optimizer)는 하지 않고, 기울기(gradient)만 구해서 출력하는 실험 코드
#
def test_training_manual_loop_no_opt(): # 함수 이름 해석; "manual_loop" → 학습을 수동으로 단계별로 진행함 (자동 훈련 아님). "no_opt" → optimizer를 실행하지 않음 (즉, 모델 파라미터는 실제로 바뀌지 않음). 대신 기울기(gradient)만 구해서 출력해 보는 테스트 코드.

    #### Skip the test on golden. It should work, need to debug why it doesn't.
    import os
    if "PYBUDA_DEVMODE" in os.environ:  # os.environ은 파이썬의 os 모듈에 포함된 환경변수 딕셔너리(dictionary). 이 딕셔너리를 통해 현재 운영체제에 설정된 환경 변수들을 조회하거나 수정할 수 있음. "PYBUDA_DEVMODE" 있으면, Golden 환경이 아니므로 나머지 코드가 실행
        pytest.skip()
    ####

    # 디바이스 및 모델 설정
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestModule("stage0")) # tt0: TT(NPU) 디바이스에 stage0이라는 테스트용 모듈을 올림.
    cpu1 = pybuda.CPUDevice("cpu1", module=pybuda.PyTorchModule("stage1", PyTorchTestModuleOneOut()))   # cpu1: CPU 디바이스에 PyTorch 기반 모듈 PyTorchTestModuleOneOut() stage1을 올림.
    cpu1.place_loss_module(pybuda.PyTorchModule("l1loss", torch.nn.L1Loss()))   # CPU 디바이스에 손실 함수(Loss function)로 L1Loss를 배치 → 예측값과 정답의 차이를 숫자로 계산
    
    # Compile & initialize the pipeline for training, with given shapes. 파이프라인 초기화 (훈련 준비)
    pybuda.initialize_pipeline(
            training=True,  # 학습(training=True) 준비를 시작
            sample_inputs=(torch.rand(4, 32, 32), torch.rand(4, 32, 32)),   # 입력 두 개 (4, 32, 32) 텐서를 넣음.
            sample_targets=(torch.rand(4, 32, 32),))    # 타겟(target, 즉 정답)도 동일한 형태의 랜덤 데이터. 이건 실제 의미 있는 데이터가 아니라 테스트용 랜덤 데이터

    steps = 2

    for step in range(steps):   # 총 2번 학습 step 반복 (step = 0, 1)
        for acc_step in range(1):   # 각 step 안에서는 1번만 accumulation (acc_step = 0)
    
            input1 = torch.rand(4, 32, 32)
            input2 = torch.rand(4, 32, 32)
            tt0.push_to_inputs((input1, input2))    # input1, input2: 임의의 입력 데이터 2개 (TT 디바이스에 넣음)

            cpu1.push_to_target_inputs(torch.rand(4, 32, 32))   # 정답 역할의 데이터 (CPU 디바이스에 넣음)

            pybuda.run_forward(input_count = 1) # 순전파 예측 수행
            pybuda.run_backward(input_count = 1, zero_grad = (acc_step == 0))   # 역전파. 손실을 기준으로 gradient 계산. zero_grad=(acc_step == 0): 처음이면 gradient 초기화

        print("Gradients on step ", step, ": ", pybuda.get_parameter_gradients())   # 계산된 기울기(gradient)를 콘솔에 출력

#
# Run training and upload new weights from host. PyBUDA에서 모델 학습은 NPU에서 실행하지만, 모델 파라미터(가중치)의 업데이트는 CPU(호스트)에서 수동으로 수행하는 예제
# 연산은 NPU(TTDevice)에서, 최적화(optimizer)는 CPU에서 직접 그러고 나서 다시 업데이트된 파라미터를 NPU로 되돌려 보내는 구조
def test_training_weight_update_on_host():

    #### Skip the test on golden. It should work, need to debug why it doesn't.
    import os
    if "PYBUDA_DEVMODE" in os.environ:  # os.environ은 파이썬의 os 모듈에 포함된 환경변수 딕셔너리(dictionary). 이 딕셔너리를 통해 현재 운영체제에 설정된 환경 변수들을 조회하거나 수정할 수 있음. "PYBUDA_DEVMODE" 있으면, Golden 환경이 아니므로 나머지 코드가 실행
        pytest.skip()
    ####

    # 모델이 두 부분으로 나뉘어 실행
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestModule("stage0")) # tt0: NPU 디바이스에 stage0 모듈 할당
    cpu1 = pybuda.CPUDevice("cpu1", module=pybuda.PyTorchModule("stage1", PyTorchTestModuleOneOut()))   # cpu1: CPU 디바이스에 stage1 모듈 할당
    cpu1.place_loss_module(pybuda.PyTorchModule("l1loss", torch.nn.L1Loss()))   # l1loss: CPU에서 손실 계산 (예측과 정답의 차이 계산)
    
    # Compile & initialize the pipeline for training, with given shapes. 실제 문제 푸는 목적이 아니라 훈련 메커니즘 실험 목적
    pybuda.initialize_pipeline(training=True,   # PyBUDA 파이프라인을 학습 모드(training=True) 로 초기화
            sample_inputs=(torch.rand(4, 32, 32), torch.rand(4, 32, 32)), # 입력 모두 랜덤값
            sample_targets=(torch.rand(4, 32, 32),)) # 정답(target)은 모두 랜덤값

    for _ in range(2):  # 2회 반복
        input1 = torch.rand(4, 32, 32) 
        input2 = torch.rand(4, 32, 32)
        tt0.push_to_inputs((input1, input2))    # 입력 2개를 TT 디바이스 푸시 

        cpu1.push_to_target_inputs(torch.rand(4, 32, 32))   # 정답 1개를 CPU 디바이스에 푸시

    # Run fwd/bwd to calculate parameter gradients
    pybuda.run_forward(input_count = 1) # run_forward: 입력을 처리하고 예측 결과 계산
    pybuda.run_backward(input_count = 1, zero_grad = True)  # run_backward: 정답과 비교해서 기울기(gradient) 계산. zero_grad=True: 이전 계산은 버리고 새로 시작

    # Retrieve weights and gradients, and use host optimizer to update weights. 여기서 중요한 건: 이제 NPU에서 계산된 gradient를 CPU로 가져왔다는 것
    grads = pybuda.get_parameter_gradients(tt0) # 각 파라미터에 대해 계산된 기울기(gradient) 값. 이 코드줄은 PyBUDA에서 제공하는 API로, 디바이스(tt0) 안의 내용을 호스트(CPU 메모리로) 복사해 가져오는 동작 임.
    params = pybuda.get_parameter_checkpoint(tt0)   # 현재 TT 디바이스에 있는 모델 파라미터(가중치) 들. 이 코드줄은 PyBUDA에서 제공하는 API로, 디바이스(tt0) 안의 내용을 호스트(CPU 메모리로) 복사해 가져오는 동작 임.
    for name in params[0]:
        params[0][name].value().grad = grads[0][name].value()   # 각 파라미터에 대응하는 gradient를 직접 할당. value()는 PyBUDA 특유의 래퍼를 벗겨서 실제 텐서로 접근하는 함수
    opt = torch.optim.SGD([p.value() for p in params[0].values()], lr=10.0) # PyTorch의 SGD 옵티마이저를 사용해서 CPU 상에서 직접 weight 업데이트. lr=10.0: 학습률 (엄청 크게 설정되어 있음 — 테스트 목적)
    opt.step()

    # Push new weights to the device
    pybuda.update_device_parameters(tt0, params)    # 앞서 CPU에서 업데이트한 파라미터를 TT 디바이스(tt0)로 다시 업로드

    # Run again with new weights. 다시 한 번 forward & backward 실행. 업데이트된 가중치로 다시 예측과 역전파 수행
    pybuda.run_forward(input_count = 1)
    pybuda.run_backward(input_count = 1, zero_grad = True)
#여기에서 얘기하고자 하는 것:
# 일반적으로는 NPU가 forward/backward/optimizer 전부 처리. 이 예제 코드는 아래를 보여주기 위함 임.
# 호스트(Host, 즉 CPU)에서도 파라미터를 제어할 수 있다.
# gradient만 TT에서 계산하고, weight update는 외부에서 커스터마이즈할 수 있다
# 즉, PyBUDA의 유연한 학습 구조를 실험하는 테스트 예제

# 
# Run inference pipeline and provide mp queues for device-to-device data. PyBUDA에서 추론(inference) 실행 시, 여러 디바이스 간 데이터를 어떻게 주고받는지(device-to-device, D2D)를 보여주는 예제.
# 특히, 각 디바이스 사이의 출력값을 mp 큐(queue)에 저장해서 나중에 확인하는 방식을 보여줌.
def test_inference_device_to_device_data(): # "추론 테스트(inference test)"를 위한 함수. 목적: 여러 디바이스 사이에서 데이터가 잘 전달되는지 확인하기 위한 테스트
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestModule("stage0")) # tt0: NPU 디바이스에 stage0 모듈 실행
    cpu1 = pybuda.CPUDevice("cpu1", module=pybuda.PyTorchModule("stage1", PyTorchTestModule())) # cpu1: CPU 디바이스에 stage1 모듈 실행
    cpu2 = pybuda.CPUDevice("cpu2", module=pybuda.PyTorchModule("stage2", PyTorchTestModuleOneOut()))   # cpu2: CPU 디바이스에 stage2 모듈 실행
    # 모델이 3단계로 분할(stage0 → stage1 → stage2) 되어 있고, 각기 다른 디바이스에 나뉘어 배치된 구조
    
    # Compile & initialize the pipeline for inference, and provide d2d mp queues to store device-to-device data in for further analysis. 디바이스 간 통신용 큐 생성 (d2d queue)
    # 각 디바이스의 출력값을 받아 저장할 큐(queue)를 생성
    # mp_context.Queue()는 Python의 멀티프로세싱 큐로, 디바이스 간 데이터를 안전하게 주고받는 통로
    tt0_output_q = mp_context.Queue()
    cpu1_output_q = mp_context.Queue()
    pybuda.initialize_pipeline(training=False, d2d_fwd_queues=[tt0_output_q, cpu1_output_q], # 파이프라인 초기화 (추론 모드 + d2d 큐 지정). training=False: 추론 모드 (학습 아님). d2d_fwd_queues: 디바이스 간 데이터 흐름을 중간에서 저장해줄 큐를 지정
            sample_inputs=(torch.rand(4, 32, 32), torch.rand(4, 32, 32) ))  # 입력 데이터의 형태(샘플)를 지정

    for _ in range(2): # 2번 수행
        input1 = torch.rand(4, 32, 32)
        input2 = torch.rand(4, 32, 32)
        tt0.push_to_inputs((input1, input2))

    # Run fwd
    pybuda.run_forward(input_count = 1) # 루프를 2번 수행했지만, 실제 실행은 1개만 처리

    # Read d2d queues
    print(_safe_read(tt0_output_q)) # tt0의 출력값 출력
    print(_safe_read(cpu1_output_q))    # cpu1의 출력값 출력

# 
# Run training pipeline and provide mp queues for device-to-device data. PyBUDA로 학습(training) 을 실행하면서, forward와 backward 도중 디바이스 간 주고받는 데이터를 큐(queue)에 저장해서 추적하는 테스트 예제. 
# 입력 데이터가 forward 방향으로 어떻게 흐르고, 기울기(gradient)가 backward 방향으로 어떻게 되돌아가는지를 확인. 디바이스끼리 데이터를 주고받는 흐름 (forward & backward)을 큐(mp.Queue)를 통해 기록하고 출력해서 분석하려는 코드.

def test_training_device_to_device_data():
    
    # 디바이스와 모델 설정
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestModule("stage0")) # tt0 (NPU); 모델의 첫 번째 단계 stage0
    cpu1 = pybuda.CPUDevice("cpu1", module=pybuda.PyTorchModule("stage1", PyTorchTestModule())) # cpu1; 두 번째 단계 stage1
    cpu2 = pybuda.CPUDevice("cpu2", module=pybuda.PyTorchModule("stage2", PyTorchTestModuleOneOut()))   # cpu2; 두 번째 단계 stage2
    cpu2.place_loss_module(pybuda.PyTorchModule("l1loss", torch.nn.L1Loss()))   # 손실 함수는 CPU에서 실행(이유: 연산량이 적기 때문(CPU에서 처리해도 성능 차이 없음, NPU 리소스를 보호(텐서 연산용으로 양보), 디버깅과 유지보수가 쉬움(CPU에서 하면 값 추적(디버깅)이 편하고 안정적))) → Cross-device(서로 다른 종류의 디바이스(예: CPU + NPU 또는 CPU + GPU)가 한 모델 파이프라인을 나눠서 실행하는 구조를 말함) 학습 파이프라인 구성
    
    # Compile & initialize the pipeline for inference, and provide d2d mp queues to store device-to-device data in for further analysis. 디바이스 간 데이터 저장용 큐 생성
    tt0_output_q = mp_context.Queue()   # tt0_output_q  forward: tt0 → cpu1 전달 값
    cpu1_output_q = mp_context.Queue()  # cpu1_output_q	forward: cpu1 → cpu2 전달 값
    cpu1_bwd_output_q = mp_context.Queue()  # cpu1_bwd_output_q	backward: cpu2 → cpu1 전달되는 gradient
    cpu2_bwd_output_q = mp_context.Queue()  # cpu2_bwd_output_q	backward: cpu2 내부에서 계산된 gradient
    
    # 파이프라인 초기화 (학습 모드)
    pybuda.initialize_pipeline(
            training=True,  # 학습 모드 (forward + backward 수행)
            d2d_fwd_queues=[tt0_output_q, cpu1_output_q],   # forward 도중 디바이스 사이 데이터를 큐에 저장
            d2d_bwd_queues=[cpu1_bwd_output_q, cpu2_bwd_output_q],  # backward 도중 디바이스 사이 gradient 데이터를 큐에 저장
            sample_inputs=(torch.rand(4, 32, 32), torch.rand(4, 32, 32)),   # 입력 두 개 (4, 32, 32) 텐서를 넣음.
            sample_targets=(torch.rand(4, 32, 32),))    # 타겟(target, 즉 정답)도 동일한 형태의 랜덤 데이터. 이건 실제 의미 있는 데이터가 아니라 테스트용 랜덤 데이터
    
    # 입력과 정답을 push (2번 반복)
    for _ in range(2):
        input1 = torch.rand(4, 32, 32)
        input2 = torch.rand(4, 32, 32)
        tt0.push_to_inputs((input1, input2))    # 랜덤 입력 2개 생성해서 TT 디바이스(tt0)에 넣음

        cpu2.push_to_target_inputs(torch.rand(4, 32, 32))   # 정답 데이터는 CPU 디바이스(cpu2)에 넣음

    # Run fwd/bwd 
    pybuda.run_forward()    # run_forward()에 input_count 안 주면 기본값이 1이므로 → 1개만 실제 실행
    pybuda.run_backward(zero_grad = True)

    # Read d2d queues. 디바이스 간 데이터 출력 (forward & backward)
    print(_safe_read(tt0_output_q)) # tt0의 출력 (cpu1 입력)
    print(_safe_read(cpu1_output_q))    # cpu1의 출력 (cpu2 입력)
    print(_safe_read(cpu1_bwd_output_q))    # cpu1로 되돌아온 gradient
    print(_safe_read(cpu2_bwd_output_q))    # cpu2의 내부 gradient
    pybuda.get_parameter_gradients(tt0)

#
# Override data formats. 데이터 포맷 강제(override) 설정 관련 테스트와 두 개의 학습률 스케줄러 클래스
#
def test_data_formats_input_override(): # 입력 및 가중치 데이터 형식을 명시적으로 설정. 이 테스트는 모듈 내부 파라미터와 입력값 모두에 대해 Float16을 강제 적용하여 추론을 실행하는 예제

    mod = PyBudaTestModule("mod")   # mod: 테스트용 모듈 생성 (PyBudaTestModule은 단순한 테스트 모듈임)
    tt0 = pybuda.TTDevice("tt0", module=mod)    # tt0: "tt0"이라는 이름의 Tenstorrent NPU 디바이스 객체 생성. 이 디바이스에 mod를 올림

    # Explicitly set data formats for parameters and inputs. weights1, weights2: mod에 포함된 가중치 변수들
    mod.weights1.set_data_format(pybuda.DataFormat.Float16) # 가중치들(weights1, weights2)의 데이터 포맷을 Float16으로 명시적으로 설정 -> 추론 속도 향상과 메모리 절약 효과 
    mod.weights2.set_data_format(pybuda.DataFormat.Float16) # 가중치들(weights1, weights2)의 데이터 포맷을 Float16으로 명시적으로 설정 -> 추론 속도 향상과 메모리 절약 효과
    input1 = torch.rand(4, 32, 32, dtype=torch.float16) # Float16 타입의 텐서 2개를 생성(batch size: 4, shape: 32x32).
    input2 = torch.rand(4, 32, 32, dtype=torch.float16) # Float16 타입의 텐서 2개를 생성(batch size: 4, shape: 32x32).
    tt0.push_to_inputs((input1, input2))    # 위에서 생성한 입력(input1, input2) 텐서를 디바이스(tt0)에 넣음.

    pybuda.run_inference()  # 추론(inference) 실행

def test_data_formats_fp32_fallback():  # 디바이스 수준에서 Float32 입력을 Float16으로 자동 변환
    
    # On this device, fall back to Float16 wherever Float32 is used
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestModule("mod"), fp32_fallback=pybuda.DataFormat.Float16)   # 이 NPU 디바이스는 Float32(단정밀도, 일반적인 torch 기본값) 입력이 들어오면 자동으로 Float16으로 변환해서 처리하도록 설정됨

    # Push Float32, which will be converted to Float16 due to fp32_fallback. Float32 형식의 입력 생성 
    input1 = torch.rand(4, 32, 32)  # Float32 타입의 텐서 2개를 생성(batch size: 4, shape: 32x32). (따로 dtype 지정 안 하면 기본이 Float32). Float32 타입으로 입력을 생성했지만, 위 설정 덕분에 내부적으로 Float16으로 처리됨
    input2 = torch.rand(4, 32, 32)  # Float32 타입의 텐서 2개를 생성(batch size: 4, shape: 32x32). (따로 dtype 지정 안 하면 기본이 Float32). Float32 타입으로 입력을 생성했지만, 위 설정 덕분에 내부적으로 Float16으로 처리됨
    tt0.push_to_inputs((input1, input2))    # 위에서 생성한 입력(input1, input2) 텐서를 디바이스(tt0)에 넣음.

    pybuda.run_inference()  # 추론(inference) 실행

def test_data_formats_op_override():    # Data Format 관련 테스트 함수. 특정 연산자 (예: matmul1)의 출력 포맷을 직접 오버라이드(Bfp8_b로 재정의) 해보는 테스트. 오버라이드; 원래 정해진 걸 내가 다시 정해서 바꾸는 것
    
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestModule("mod"))    # 디바이스 초기화 및 모듈 할당

    # Use API to set manual data format override on an op
    pybuda.configure_mixed_precision(name_regex="matmul1", output_df=pybuda.DataFormat.Bfp8_b)  # matmul1이라는 이름을 가진 연산자의 출력 데이터 포맷을 Bfp8_b로 바꾸겠다는 뜻. Bfp8_b는 Block Floating Point 8-bit 형식 중 하나 (성능 최적화에 유용)
    
    input1 = torch.rand(4, 32, 32)  # 배치 크기 (Batch size) → 입력 샘플 4개를 한 번에 처리하겠다는 뜻. 각 텐서 크기 32X32
    input2 = torch.rand(4, 32, 32)  # 배치 크기 (Batch size) → 입력 샘플 4개를 한 번에 처리하겠다는 뜻. 각 텐서 크기 32X32
    tt0.push_to_inputs((input1, input2))    # input 값 디바이스(tt0)에 로딩

    pybuda.run_inference()  # 추론 실행

class TorchSchedulerWithWarmupAndDecay(pybuda.torch_schedulers.TorchLearningRateScheduler): # Learning Rate Scheduler 클래스. TorchSchedulerWithWarmupAndDecay; PyBUDA가 아니라 PyTorch의 옵티마이저 스케줄러를 상속한 클래스.
    def __init__(self, optimizer):
        super().__init__(optimizer)
    
    def get_lr(self):
        return [self.optimizer.param_groups[0]["lr"] + 1]   # 현재 학습률에 +1을 한 값을 반환 (실제 스케줄링 로직은 단순화됨, 테스트용)
    
    def step(self):
        super().step()
        print(f"Torch optimizer learning rate updated to {self.optimizer.param_groups[0]['lr']}")   # 매 step마다 PyTorch 옵티마이저의 학습률을 업데이트하고 출력함


class TestScheduler(LearningRateScheduler): # TestScheduler: PyBUDA 전용 학습률 스케줄러. PyBUDA에서 직접 사용하는 옵티마이저 학습률 스케줄러 정의
        def __init__(self, optimizer):
            super().__init__(optimizer)
        
        def get_lr(self):
            return self.optimizer.learning_rate + 1 # 현재 PyBUDA 옵티마이저의 학습률에 +1을 한 값 리턴. 이 코드는 테스트 용도로 작성이 되어서 그런지 +1을 해 주었는데, 실전에서는 보통 감소(decay) 방식의 스케줄링을 사용 (이유: 하기 학습률/스케줄링 개념 참고)
        
        def step(self):
            super().step()
            print(f"Pybuda optimizer learning rate updated to {self.optimizer.learning_rate}")  # 매 스텝마다 PyBUDA 옵티마이저의 학습률 출력
        
        def get_pytorch_scheduler(self, optimizer: torch.optim.Optimizer):  # PyTorch 옵티마이저를 입력받아 위에서 정의한 TorchSchedulerWithWarmupAndDecay를 생성하여 연결 -> CPU 기반 디바이스에 PyTorch 옵티마이저를 사용하는 경우 필요
            if self.torch_scheduler is None:
                self.torch_scheduler = TorchSchedulerWithWarmupAndDecay(
                    optimizer=optimizer
                )
            
            return self.torch_scheduler
# 학습률(Learing Rate)
# 딥러닝에서 학습이란, 어떤 최적점(최소값)을 찾아가는 과정
# 이건 마치 언덕을 내려가서 골짜기 바닥을 찾는 일
# 우리가 찾는 건 손실 함수(loss function)의 최솟값(=골짜기 바닥)
# 딥러닝 모델이 학습할 때 얼마나 빠르게 가중치를 바꿀지를 결정하는 숫자
# 학습률이 너무 크면: 너무 크게 바꿔서 진동하거나 학습 실패. 진동; 학습률이 너무 크면, 반대쪽 언덕으로 훅 넘어가버림, 다시 돌아와도 반대쪽(원래대로)으로 훅 넘어가버림. 이렇게 왔다갔다 하는 것을 진동이라고 함.
# 학습률이 너무 작으면: 느리게 학습하거나, 최적점에 도달하기 전에 멈출 수 있음. (멈추는 이유: 이유1. 최적점 근처에서 멈춰버림. 학습률이 너무 작으면, 손실(loss)의 변화도 너무 작게 느껴져서 옵티마이저(최적화기)가 “아~ 거의 다 왔네?” 하며 학습을 멈추는 조건(early stopping, gradient가 거의 0 등)에 걸릴 수 있음. 이유2.부동소수점 한계 때문에 멈춤. 컴퓨터는 숫자를 유한한 정밀도(float32 등)로 계산. 학습률이 너무 작으면, 업데이트 값이 너무 작아서 계산 결과가 0처럼 무시될 수 있음. 이유3.학습 종료 조건(early stopping, max epoch)에 걸림. 딥러닝 학습은 보통 일정 횟수(max_epochs)나 성능 개선이 없을 때 종료. 너무 느리면, 최소값에 도달하기 전에 시간이 다 돼서 종료될 수 있음)
# 스케줄링(Scheduling)
# 학습 초반에는 크게 움직이고
# 학습 후반에는 조금씩 미세하게 움직이는 게 더 좋음
# 이걸 자동으로 조절하는 게 학습률 스케줄링(Learning Rate Scheduling)


# Run the learning rate scheduler across 100 steps to
# show how optimizer learning rate gets updated.
# 학습률 스케줄러(learning rate scheduler)가 100번의 단계 동안 어떻게 학습률을 바꾸는지 확인하는 테스트
def test_learning_rate_scheduler(): # test_learning_rate_scheduler 라는 이름의 테스트 함수 정의. 이 함수를 통해 학습 스케줄러가 잘 동작하는지 확인.
            
    lr = 1  # 학습률(learning rate)을 1로 설정. 학습 속도를 조절하는 아주 중요한 하이퍼파라미터. 학습 속도의 중요성에 대한 내용은 상기 내용 참고.
    optimizer = pybuda.optimizers.SGD(learning_rate=lr, device_params=True) # SGD; Stochastic Gradient Descent (확률적 경사 하강법)의 줄임말. 딥러닝에서 가장 기본적인 최적화 방법. 오차를 줄이는 방향(gradient 방향)으로 조금씩 파라미터를 조정해 가며 학습. learning_rate=1; 얼마나 많이 움직일지 결정하는 숫자. 값이 크면 빨리 움직이지만, 너무 크면 목적지를 지나칠 수 있음. 값이 작으면 안정적이지만 느림. 여기서 0.1은 다소 빠르게 파라미터를 조정하는 설정. device_params=True; 모델의 파라미터(weight 등)를 NPU 디바이스(tt0) 안에 올려서 관리하겠다는 뜻. 기본적으로 PyTorch는 CPU나 GPU에서 파라미터를 관리. True로 지정하면, 파라미터가 NPU에 직접 올라가서 연산 및 업데이트가 거기서 일어남. NPU에서 모든 연산을 효율적으로 처리할 수 있어서 속도 및 대역폭 절감 효과가 나타남.
    scheduler = TestScheduler(optimizer=optimizer)  # TestScheduler 객체를 생성. 이 스케줄러는 옵티마이저를 받아서 학습률을 자동으로 조절해주는 역할. 'optimizer=optimizer' 표기는 '옵티마이저는 상기에 코드된 'optimizer = ...''다 라는 소리임. 그냥 optimizer라고 표기해도 되지만, 다른 개발자의 가독성 높이기 위해 이렇게 함.(헷갈리지 않고, 읽기 쉬움)
    
    tt0 = pybuda.TTDevice(  # Tenstorrent NPU 디바이스를 생성
        "tt0",  # 장치 이름
        module=PyBudaTestModuleOneOut("stage0"),    # PyBudaTestModuleOneOut("stage0"): 테스트용 모듈을 로딩 (출력이 하나인 모듈)
        optimizer=optimizer,    # 학습 관련 구성요소(optimizer) 전달
        scheduler=scheduler     # 학습 관련 구성요소(scheduler) 전달
    )
    cpu1 = pybuda.CPUDevice(    # CPU 디바이스 생성
        "cpu1", # "cpu1"이라는 이름의 장치
        module=pybuda.PyTorchModule(    # PyTorch 모듈을 하나 올림
            "stage1",
            PyTorchTestModuleOneInputAndOneOut()    # 이 모듈은 입력 하나, 출력 하나를 가지는 테스트용 모듈 사용
        ),
        optimizer_f=lambda module: torch.optim.SGD(module.parameters(), lr=lr), # CPU에서 사용할 옵티마이저를 람다 함수로 정의. 옵티마이저는 PyTorch의 SGD
        scheduler_f=lambda optimizer: scheduler.get_pytorch_scheduler(optimizer=optimizer)    # CPU에서 사용할 스케줄러 람다 함수로 정의. 스케줄러는 TestScheduler에서 제공하는 PyTorch 버전
    )
    cpu1.place_loss_module( # 손실 함수(loss function)를 CPU에 등록
        pybuda.PyTorchModule(
            "loss",
            PyTorchLoss()   # PyTorchLoss()는 테스트용 손실 함수
        )
    )

    sequential = True   # 실행 순서를 순차적으로 처리하도록 설정
    pybuda.initialize_pipeline(training=True,   # 전체 파이프라인을 초기화. 학습을 위한 전체 시스템 준비 단계. training=True: 학습 모드
            sample_inputs=(torch.rand(4, 32, 32), torch.rand(4, 32, 32)),  # 입력 샘플을 미리 제공해주는 부분. 입력 텐서는 2개. 각 텐서는 4 x 32 x 32 크기. 4는 배치(Batch) 크기. PyBUDA가 이걸 보고 모델 구조에 맞게 내부 계산 그래프를 자동 준비
            sample_targets=(torch.rand(4, 32, 32),), _sequential=sequential)    # 출력(목표값, 타깃, 레이블) 샘플을 미리 제공해주는 부분. 보통 손실 함수를 위해 필요. 모델이 예측한 결과와 비교해서 얼마나 틀렸는지 계산하려면 정답(타깃)이 필요하기 때문. 튜플(자료묶음. 여러 값을 하나로 묶어서 저장할 때 사용)로 감싸져 있고((4, 32, 32)), 여기선 타깃(정답) 하나만 제공 중. _sequential=sequential: 여러 디바이스(CPU/NPU)가 있을 때 순차적으로 실행할지를 결정. 순차 실행이면 stage0 → stage1 → stage2 이런 식으로 차례차례 실행. 병렬(parallel)로 실행할 수도 있지만, 테스트에선 보통 sequential=True를 많이 사용.

    for _ in range(100):    # '100번 반복하면서 학습률 스케줄러를 실행한다'는 의미. 0부터 99까지, 총 100번 반복하라는 뜻. for _ in ...: → 반복은 하지만, 변수 이름을 안 쓸 거야 라는 뜻으로 _를 씀. '_' 아니라 'i'를 사용할 때는, 사용 숫자(i)를 써먹을 때가 있을 사용. 하지만 지금은 숫자를 안 쓰고 반복만 하고 싶음. 그래서 그냥 아무 의미 없는 _ 를 넣음.
        pybuda.run_schedulers(sequential)   # run_schedulers(...) → 학습률 스케줄러를 실행하는 함수. (sequential) → 앞에서 sequential = True 라고 했으니까 run_schedulers(True) 와 같은 뜻.
    
    
    
def test_specific_chip_id():    # test_specific_chip_id라는 이름의 테스트 함수 정의. 이 함수는 PyBUDA가 여러 칩(NPU) 중 특정 칩에서만 실행되게 할 수 있는지 테스트.
    """
    Run inference on a specific chip on a multi-chip system # 여러 개의 NPU(칩)가 있을 때, 그중 특정한 하나의 칩(마지막 칩) 에만 모델을 올려서 추론(inference) 을 해보는 테스트
    """
    num_devices = len(pybuda.detect_available_devices())    # 현재 시스템에 연결된 Tenstorrent 디바이스 개수(NPU 수) 를 확인. detect_available_devices()는 연결된 칩 리스트를 주고, len(...)은 그 리스트의 길이 = 칩 개수를 반환.

    if num_devices < 2: # 만약 칩이 1개뿐이라면, 이 테스트는 할 수 없기 때문에 건너뜀. 
        pytest.skip("Need at least 2 devices to run chip-id test")  # 실제 예. SKIPPED [1] test_file.py:45: Need at least 2 devices to run chip-id test

    input1 = torch.rand(4, 32, 32)  # 임의의 입력 데이터를 2개 만듬. 각각 4 x 32 x 32 크기의 랜덤 텐서 (배치 4개짜리, 32x32 크기 데이터)
    input2 = torch.rand(4, 32, 32)

    # Create a TT device, on last available chip
    tt0 = pybuda.TTDevice("tt0", chip_ids=[num_devices-1])  # TTDevice 객체를 하나 만들고, 그걸 마지막 칩(chip_ids[-1]) 에만 올림. 예를 들어 3개 칩이 있다면 num_devices = 3 (0, 1, 2)이고, chip_ids = [2]가 되어 → 세 번째 칩에서만 실행되게 됨.

    # Place a module on the device
    tt0.place_module(PyBudaTestModule("last_chip")) # "last_chip"이라는 이름을 가진 Tenstorrent 디바이스 tt0에 테스트용 모델을 하나 올림. 

    # Push intputs to the device
    tt0.push_to_inputs((input1, input2))    # 앞에서 만든 두 개의 입력 텐서를 디바이스에 전달 함. 이 입력값들이 모델에 들어가서 계산되게 됨.

    # Run pipeline, and read the outputs
    output_q = pybuda.run_inference()   # 추론(inference) 실행. 계산 결과는 output_q라는 큐(queue)에 담김.
    output = _safe_read(output_q)   # 큐에서 결과를 꺼내서 읽음.
    print(output)   # 최종 결과 화면에 출력

# “지정한 번호의 NPU 칩 하나에서만 모델을 실행해서 결과를 출력”하는 코드
def _run_on_chip(chip_id: int): # _run_on_chip이라는 이름의 함수. chip_id: int -> 파이썬에게 알려주는 힌트. chip_id는 정수(int) 형이 들어올 거야라는 소리. 실제로 이 코드는 실행할 때는 영향을 안 줌. 하지만 나중에 코드 분석 도구, IDE(편집기), Linter가 이 정보를 보고 도와줌.

    # Each process needs to have its own temporary dir
    pybuda.set_configuration_options(backend_output_dir=f"tt_build/test_out_chip_{chip_id}")    # 파이프라인 실행 결과(컴파일된 파일 등)를 저장할 폴더 위치를 정해줌. 예: chip_id=2면 → tt_build/test_out_chip_2 폴더가 생김. 이건 병렬로 여러 칩을 돌릴 때, 서로 다른 폴더를 써야 충돌이 안남.

    # 입력 데이터를 2개 생성
    input1 = torch.rand(4, 32, 32)  # 입력 데이터 생성. 크기는 4 x 32 x 32 (배치 4개짜리, 32x32 크기의 데이터)
    input2 = torch.rand(4, 32, 32)  # 입력 데이터 생성. 크기는 4 x 32 x 32 (배치 4개짜리, 32x32 크기의 데이터)

    # Create a TT device, on last available chip
    tt0 = pybuda.TTDevice("tt0", chip_ids=[chip_id])    # chip_id에 해당하는 특정 NPU 칩 하나만 사용하는 디바이스를 만듬.

    # Place a module on the device
    tt0.place_module(PyBudaTestModule(f"chip_{chip_id}"))   # 해당 칩에 모델을 올림. 모델 이름은 "chip_2"처럼 chip_id를 이용해 구분함

    # Push intputs to the device
    tt0.push_to_inputs((input1, input2))    # 앞에서 만든 입력 데이터를 해당 칩에 넣음.

    # Run pipeline, and read the outputs
    output_q = pybuda.run_inference()   # 추론(inference)을 실행. 결과는 큐(queue)에 담김. 
    output = _safe_read(output_q)   # 큐에서 결과를 꺼내 읽음. _safe_read()는 큐가 비었는지 확인하며 안전하게 꺼내는 함수
    print("From chip ", chip_id, ":", output)   # 어떤 칩에서 나왔는지, 그리고 그 결과(output)를 화면에 출력

    # Clean up the process so we can end it cleanly
    pybuda.shutdown()   # PyBUDA 시스템을 깔끔하게 종료. 이걸 해줘야 다음에 다시 초기화할 때 문제가 안 생김.

# 이 함수는 여러 개의 NPU 칩에서 동시에(병렬로) 모델을 실행하는 테스트. 즉, 칩이 4개 있으면 → 4개 모두 한 번에 돌려서 계산해보는 실험
def test_parallel_chips():  # test_parallel_chips()라는 이름의 테스트 함수. 여러 칩에서 병렬(parallel)로 실행하는 테스트
    """
    Run different models on multiple chips at the same time. 각 다른 모델을 동시에 여러 칩에 돌려본다는 테스트
    """
    pytest.skip("Appears to hang now")
    num_devices = len(pybuda.detect_available_devices())    # 사용할 수 있는 NPU 장치 개수를 세는 코드. 예: 칩이 4개 연결돼 있으면 num_devices = 4

    if num_devices < 2: # 칩이 2개 미만이면 (즉, 0개 또는 1개) → 테스트할 수 없으므로 건너뜁니다.
        pytest.skip("Need at least 2 devices to run parallel chip test")

    procs = []  # procs라는 리스트. 여기에 나중에 여러 개의 실행 중인 작업들(프로세스)을 저장
    for i in range(num_devices):    # 칩 개수만큼 반복. 예: 칩이 4개라면 i는 0, 1, 2, 3
        p = mp_context.Process(target=_run_on_chip, args=(i,))  # i번째 칩에 대해 새로운 실행 작업(process) 을 만듬. target=_run_on_chip 함수 실행. args=(i,) ➜ 이때 chip_id = i 로 넘겨줌. 예: i가 2면 ➜ chip_id=2인 칩에서 모델 실행
        p.start()   # 방금 만든 실행 작업을 시작. 즉, 그 칩에서 모델을 실제로 돌리기 시작.
        procs.append(p) # 실행된 프로세스를 procs 리스트에 저장. 나중에 전부 끝났는지 확인하려고 저장해두는 것.

    for i, p in enumerate(procs):   # 모든 실행 작업이 끝날 때까지 기다림.
        p.join()    # join()은 “다 끝날 때까지 기다리기” 기능

# Tenstorrent NPU에서 모델을 컴파일해서 파일(이미지)로 저장하고, 그 파일을 다시 불러와서 추론(inference)이 잘 되는지 테스트하는 함수
def test_tti_inference_save_and_load(): # tti 추론 저장과 불러오기를 테스트 함수
    available_devices = pybuda.detect_available_devices()   # 사용 가능한 NPU 장치들을 검색해서 리스트로 가져옴. 예: [Grayskull, Wormhole_B0] 같은 형태
    if available_devices and available_devices[0] == BackendDevice.Grayskull:   # 만약 첫 번째 장치가 Grayskull이라는 아키텍처이면...
        tt0 = pybuda.TTDevice(
            "tt0",
            arch=BackendDevice.Grayskull,
            devtype=BackendType.Golden,
        )   # Grayskull 칩을 쓰는 디바이스를 하나 만듬
    else:
        tt0 = pybuda.TTDevice(  # 그렇지 않으면 Wormhole_B0 아키텍처를 쓰는 디바이스를 생성. 즉, 어떤 NPU 칩이 연결돼 있는지에 따라 알아서 골라주는 코드
            "tt0",
            arch=BackendDevice.Wormhole_B0,
            devtype=BackendType.Golden,
        )


    module = PyBudaTestModule("test_pybuda_module") # 간단한 테스트용 모델 하나를 만듭니다. 이름은 "test_pybuda_module"
    tt0.place_module(module)    # 위에서 만든 모델을 tt0 디바이스에 올림

    # Saving to Archive
    input_shape = (1, 1, 32, 32)    # 입력으로 사용할 1x1x32x32 크기의 랜덤 텐서를 2개 만듬.(1개의 이미지, 32x32 픽셀짜리)
    input1, input2  = torch.rand(*input_shape), torch.rand(*input_shape)
    device_img = tt0.compile_to_image(  # 디바이스에 올라간 모델을 실제로 컴파일해서 .tti 파일로 저장
        img_path="device_images/test_tt0.tti",  # "device_images/test_tt0.tti"라는 이름으로 저장
        training=False, # training=False는 추론 모드로 컴파일한다는 뜻
        sample_inputs=(input1, input2), # sample_inputs는 예시 입력값. 여기까지가 "모델 저장(save)" 단계
    )
    pybuda_reset()  # flush the global state that lingers around for test. pybuda_reset() PyBUDA의 전체 상태를 초기화(reset). 디바이스, 모델, 설정 등 모든 걸 리셋해서 불러오기(load) 테스트가 완전히 독립되게 하기 위한 준비

    # Loading from Archive. 
    tt1 = pybuda.TTDevice.load_image(img_path="device_images/test_tt0.tti") # .tti 파일을 읽어서 새로운 디바이스 tt1을 만듬.
    tt1.push_to_inputs((input1, input2))    # 이전에 만든 입력값을 다시 디바이스에 넣음.
    output_q = pybuda.run_inference()   # 추론(inference) 실행
    output = _safe_read(output_q)   # 결과 큐에서 안전하게 값을 꺼냄.

# PyBUDA에서 특정 연산 사이에 "NOP" 연산(아무 것도 하지 않는 명령) 을 넣는 기능이 잘 작동하는지 확인하는 테스트. NOP이 필요한 이유; 컴퓨터는 정확한 순서, 타이밍이 아주 중요. 그런데 어떤 연산(계산) 사이에 "시간을 조금 벌어야 한다"거나 "자리 정렬이 필요하다"**는 상황이 있음. 계산 A → (아무것도 안 하는 NOP) → 계산 B; NOP을 하나 껴 넣으면, 시간을 살짝 벌거나, 연산 경로가 깔끔하게 정리됨.
@pytest.mark.parametrize("hoist_tms", [True, False])    # @의 의미; 파이썬에서 "데코레이터(decorator)" 라고 불리는 문법. 함수 앞에 붙여서, 그 함수의 동작을 살짝 바꾸거나 확장해주는 도구. @pytest.mark.parametrize(...) 는 바로 아래 함수인 test_nop_insertion_api()에 추가 기능을 붙여주는 역할. PyTest에게 같은 테스트 함수를 2번 실행하라고 알려주는 줄. 매번 hoist_tms 값을 다르게 넣어서 실행. test_nop_insertion_api(hoist_tms=True), test_nop_insertion_api(hoist_tms=False) 이렇게 2번 실행. hoist_tms는 PyBUDA에서 연산 중 템포럴 스케줄(TMS)을 위로 끌어올릴지 말지를 정하는 옵션. 지금은 True일 때와 False일 때 둘 다 잘 작동하는지 확인하는 게 목적
def test_nop_insertion_api(hoist_tms):  # 테스트 함수가 정의. hoist_tms는 위에서 넘겨준 True 또는 False 값을 받음.
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestQueryKeyModule(f"query_key_module_hoist_tms_{hoist_tms}"))    # Tenstorrent 디바이스 tt0를 만들고, 테스트용 모듈(PyBudaTestQueryKeyModule) 을 올림. 모듈 이름은 "query_key_module_hoist_tms_True" 또는 "query_key_module_hoist_tms_False"가 됨.

    # Use API to set manual data format override on an op. "NOP 연산"을 수동으로 삽입하는 명령
    pybuda.insert_nop("mha_key", "mha_as", hoist_tms=hoist_tms) # "mha_key" → 원래 연산의 이름, "mha_as" → 그 다음 연산의 이름. 그 사이에 NOP (아무것도 안 하는 연산)를 끼워 넣음. hoist_tms=True 또는 False에 따라, 삽입 위치가 달라짐. NOP 삽입이 뭔가? NOP을 내가 원하는 위치에 일부러 넣는 것. pybuda.insert_nop("앞 연산", "뒷 연산") -> 이렇게 쓰면: 앞 연산과 뒷 연산 사이에 빈 연산(NOP) 이 하나 들어감.
    microbatch_size, seq_len, hidden_dim = (1, 128, 128)    # 입력 텐서를 하나 만듬. 크기는 [1, 128, 128]이고, 이는 Transformer 모델의 시퀀스 입력 모양(Transformer가 좋아하는 입력 모양과 거의 똑같다는 뜻). batch (microbatch_size)/1/문장 1개 (또는 데이터 1개), sequence_length (seq_len)/128/단어가 128개 있음. hidden_dim/128/단어 하나를 숫자 128개로 표현함
    encoder_input = torch.rand(microbatch_size, seq_len, hidden_dim)
    # [
    #   [벡터128개]  ← 1번째 단어
    #   [벡터128개]  ← 2번째 단어
    #   ...
    #   [벡터128개]  ← 128번째 단어
    # ]

    tt0.push_to_inputs((encoder_input)) #  만든 입력 텐서를 디바이스 tt0에 넣음.
    pybuda.run_inference()  # 추론(inference)을 실행. 이 과정에서 NOP(No Operation; 아무 일도 안 하는 명령어) 삽입이 문제 없이 반영되어 동작하는지 확인

@pytest.mark.parametrize("hoist_tms", [True, False])
def test_nop_fork_insertion_api(hoist_tms):
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestQueryKeyModule(f"forking_nop_insertion{hoist_tms}"))

    # Use API to set manual data format override on an op
    pybuda.insert_nop("encoder_input", ["mha_key", "mha_query"], hoist_tms=hoist_tms)
    microbatch_size, seq_len, hidden_dim = (1, 128, 128)
    encoder_input = torch.rand(microbatch_size, seq_len, hidden_dim)

    tt0.push_to_inputs((encoder_input))
    pybuda.run_inference()

@pytest.mark.parametrize("hoist_tms", [True, False])
def test_nop_daily_chain_insertion_api(hoist_tms):
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestForkWithThreeUsers(f"daisy_chain_nop_insertion{hoist_tms}"))

    # Use API to set manual data format override on an op
    pybuda.insert_nop("encoder_input", ["mm_a", "mm_b", "mm_c"], hoist_tms=hoist_tms)
    pybuda.insert_nop("buffer_0_encoder_input_mm_a", ["mm_b", "mm_c"], hoist_tms=hoist_tms)
    pybuda.insert_nop("buffer_0_buffer_0_encoder_input_mm_a_mm_b", ["mm_c"], hoist_tms=hoist_tms)
    microbatch_size, seq_len, hidden_dim = (1, 128, 128)
    encoder_input = torch.rand(microbatch_size, seq_len, hidden_dim)

    tt0.push_to_inputs((encoder_input))
    pybuda.run_inference()

def test_dram_channel_override():
    tt0 = pybuda.TTDevice("tt0", module=PyBudaTestModule(f"dram_channel_override"))

    # Use API to set manual data format override on an op
    input1 = torch.rand(4, 32, 32)
    input2 = torch.rand(4, 32, 32)
    pybuda.config.override_dram_queue_placement("e2e_matmul1_0", channel=0)
    pybuda.config.set_epoch_break("matmul2")

    tt0.push_to_inputs((input1, input2))
    pybuda.run_inference()

@pytest.mark.parametrize("loss", ["l1", "mse"])
def test_loss_module_on_ttdevice(loss):
    import torch.nn as nn
    class Lin(nn.Module):
        def __init__(self, d_model):
            super(Lin, self).__init__()
            self.input_linear = nn.Linear(1, d_model)

        def forward(self, src):
            output = self.input_linear(src)
            return output

    model = Lin(1)
    tt0 = pybuda.TTDevice(
        "tt0",
        module=pybuda.PyTorchModule("lin", model),
        optimizer=pybuda.optimizers.SGD(learning_rate=0.1, device_params=True)
    )
    if loss == "mse":
        tt0.place_loss_module(pybuda.PyTorchModule("mse_loss", nn.MSELoss()))
    else:
        tt0.place_loss_module(pybuda.PyTorchModule("l1_loss", nn.L1Loss()))

    inputs = torch.rand(1, 1)
    targets = torch.rand(1, 1)

    # Initialize pipeline
    checkpoint_q = pybuda.initialize_pipeline(
       training=True,
       sample_inputs=(inputs,),
       sample_targets=(targets,)
    )

    tt0.push_to_inputs(inputs)
    tt0.push_to_target_inputs(targets)
    pybuda.run_forward(input_count=1)
    pybuda.run_backward(input_count=1, zero_grad=True)
    pybuda.run_optimizer(checkpoint=True)